In [ ]:
%matplotlib inline

from collections import defaultdict
import functools
import os
import urllib

from IPython.core.display import HTML
from Bio import SeqIO
import numpy as np
import pandas as pd
import pybedtools
import pysam
import seaborn as sns
import matplotlib
from matplotlib import gridspec
import scipy
from tqdm import tqdm, tqdm_notebook
tqdm.pandas("Progress: ")

from gscripts.general import dataviz
from gscripts.encode import encode_helpers
from gscripts import qtools

In [ ]:
img_dir = "/home/gpratt/Dropbox/encode_integration/qc_work/"

sns.set_style("ticks")
xfmt = matplotlib.ticker.FuncFormatter(lambda x, p: format(float(x) / 1000000, ','))
xfmt_int = matplotlib.ticker.FuncFormatter(lambda x, p: format(int(float(x) / 1000000), ','))

RESET = False

#reads = "family_reads"
reads = "reads"

unique = "unique"


In [ ]:
import matplotlib as mpl
legend = None

OUTSIDE_LEGEND_SAVEFIG_KWS = dict(bbox_extra_artists=(legend,),
                                  bbox_inches='tight')
from matplotlib import rc

mpl.rcParams['svg.fonttype'] = 'none'

rc('text', usetex=False) 
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

In [ ]:
out_dir = "/home/gpratt/projects/encode/analysis/peak_reanalysis_v14/"
downsample_path = "/home/gpratt/projects/idr/analysis/downsample_v2/"
ew_input_norm_dir = "/home/elvannostrand/data/clip/CLIPseq_analysis/EW_v9_2015210"

In [ ]:
merged_data = encode_helpers.get_merged_data()

#For Peak Analysis we only want to analyze datasets that have been submitted
merged_data = merged_data[merged_data.submitted]
merged_data = merged_data[['CLIP', 'INPUT', 'input_norm']]

In [ ]:
len(merged_data) / 2.

181.0

# Generate Downsampling file

In [ ]:
#Ignoring this for now, don't need it
#Add Emily and Erics data to the dataframe for processing
# rep1 = "/projects/ps-yeolab3/encode/analysis/encode_master/EW2_CLIP_U2AF1_V5.merged.r2.bam"
# rep2 = "/projects/ps-yeolab3/encode/analysis/encode_master/EW3_CLIP_U2AF1_V5.merged.r2.bam"
# input_fn = "/projects/ps-yeolab3/encode/analysis/encode_master/EW1_INPUT_ATTACTCG-TATAGCCT_L007_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam"
# out_list = [rep1, rep2, input_fn, "hg19", "EW"]
# out_file.write("\t".join(out_list) + "\n")

# submitted_datasets.loc[181] = ["EW", "U2AF1_V5", "HepG2", rep1, rep2, input_fn]
# submitted_datasets.loc[182] = ["293XT_CLIP_RBFOX2_0204", "RBFOX2", "293T", 
#                            "/projects/ps-yeolab3/encode/analysis/encode_v12/293XT_CLIP_RBFOX2_0204_RBFOX2.merged.r2.bam", 
#                            "/projects/ps-yeolab3/encode/analysis/encode_v12/293XT_CLIP_RBFOX2_0204_RBFOX2.merged.r2.bam",
#                            "/projects/ps-yeolab3/encode/analysis/encode_v12/293XT_Input_0204_ATTCAGAA-TAATCTTA_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam"]
# submitted_datasets.loc[183] = ["293XT_CLIP_RBFOX2_1120", "RBFOX2", "293T", 
#                            "/projects/ps-yeolab3/encode/analysis/encode_v12/293XT_CLIP_RBFOX2_1120_RBFOX2.merged.r2.bam", 
#                            "/projects/ps-yeolab3/encode/analysis/encode_v12/293XT_CLIP_RBFOX2_1120_RBFOX2.merged.r2.bam", 
#                            "/projects/ps-yeolab3/encode/analysis/encode_v12/293XT_Input_1120_GAATTCGT-GGCTCTGA_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam"]

In [ ]:
with open("/home/gpratt/projects/idr/scripts/downsample_v2.txt", 'w') as out_file:
#     out_list = [rep1, rep2, input_fn, "hg19", "EW"]
#     out_file.write("\t".join(out_list) + "\n")

    for name, row in merged_data.unstack().iterrows():
        uID = name[0]
        out_list = [row[('CLIP', "rep1")], row[('CLIP', "rep2")], row[('INPUT', "rep1")], "hg19", uID]
        out_file.write("\t".join(out_list) + "\n")

In [ ]:
# #Converts reps into dataframe, like they should be
# submitted_datasets = submitted_datasets.set_index(["uID", "RBP", "Cell line", "INPUT"])
# submitted_datasets.columns = pd.MultiIndex.from_tuples([item.split("_") for item in submitted_datasets.columns])
# submitted_datasets = submitted_datasets.stack()
# submitted_datasets['INPUT'] = submitted_datasets.index.get_level_values(level="INPUT")
# submitted_datasets.index = submitted_datasets.index.droplevel("INPUT")

Format to replicate for downsampling

ip 1\t ip 2 \t input \t species \t id


/projects/ps-yeolab3/encode/analysis/encode_v12/213_02_RBM34.merged.r2.bam      /projects/ps-yeolab3/encode/analysis/encode_v12/231_01_RBM34.merged.r2.bam      /projects/ps-yeolab3/encode/analysi
s/encode_v12/213_INPUT_ATTCAGAA-ATAGAGGC_L002_R1.unassigned.adapterTrim.round2.rmRep.rmDup.sorted.r2.bam     hg19    213  


# Plot Peak Level downsampling and correlations

In [ ]:
make_and_filter_clipper_moderate = functools.partial(encode_helpers.make_and_filter_clipper, l2fc=3, pval=3, reset=False)

In [ ]:
#Get all downsampled datasets and also put them into a tidy dataframe
merged_data = merged_data.set_index("INPUT", append=True)

merged_data.columns = pd.MultiIndex.from_tuples([("CLIP", 1.0), ("input_norm", 1.0)], names=["input_norm", "fraction"])

for percent in range(1,10):
    percent_downsample = "0{}".format(percent)
    merged_data[('CLIP', percent / 10.)] = merged_data[("CLIP", 1.0)].apply(lambda x: downsample_path + os.path.basename(x).replace(".bam", ".{}.bam".format(percent_downsample)))
    merged_data[('input_norm', percent / 10.)] = merged_data[("CLIP", 1.0)].apply(lambda x: downsample_path + os.path.basename(x).replace(".bam", ".{}.peaks.norm.compressed.bed".format(percent_downsample)))

merged_data = merged_data.stack()
merged_data = merged_data.reset_index()
merged_data = merged_data.set_index([u'uID',u'RBP', u'Cell line', "rep", u'fraction'])
merged_data = merged_data.sort_index()

In [ ]:
#ON RESET FIX THIS TO BE INLINE WITH EVERYTHING ELSE, UNTIL THEN FUCK IT
foo = merged_data.copy()
ect_and_usable = pd.read_csv("ect_estimate_usable_reads.csv", index_col=[0,1,2,3])
ect_and_usable = ect_and_usable[['uncollapsed', 'unique', 'eCT', 'fraction_usable', 'adjusted_ect_concentration']]
ect_and_usable = ect_and_usable.reset_index()
ect_and_usable['rep'] = ect_and_usable.bio_rep.apply(lambda x: "rep1" if x == 1 else "rep2")
initial_counts = pd.merge(foo, ect_and_usable, left_on=['uID', 'RBP', 'rep', 'fraction'], right_on=['RBP_ID', 'RBP', 'rep', 'fraction'])
initial_counts = initial_counts.set_index(["uID", 'RBP', 'Cell line', 'rep', 'fraction'])

KeyError: 'uID'

In [ ]:
merged_data['filtered_moderate'] = merged_data['input_norm'].progress_apply(make_and_filter_clipper_moderate)

In [ ]:
def bedtools_from_row(row):
    total_bedtool = total_bedtools.ix[row.name[:-1]].filtered_moderate
    fractional_bedtool = row.filtered_moderate
    
    total_bedtool = pybedtools.BedTool(total_bedtool)
    fractional_bedtool = pybedtools.BedTool(fractional_bedtool)
    return fractional_bedtool, total_bedtool

def fraction_bound(row):
    fractional_bedtool, total_bedtool = bedtools_from_row(row)
    return len(total_bedtool.intersect(fractional_bedtool, u=True, s=True, sorted=True)) / float(len(total_bedtool))
 
def fraction_of_fraction_in_total(row):
    fractional_bedtool, total_bedtool = bedtools_from_row(row)
    return len(fractional_bedtool.intersect(total_bedtool, u=True, s=True, sorted=True)) / float(len(fractional_bedtool))

def fraction_not_in_total(row):
    fractional_bedtool, total_bedtool = bedtools_from_row(row)
    return len(fractional_bedtool.intersect(total_bedtool, v=True, s=True, sorted=True)) / float(len(fractional_bedtool))

def get_entropy_from_annotated(fn):
    fn = os.path.basename(fn)
    stripped_fn = ".".join(fn.split(".")[:-3])
    stripped_fn = os.path.join(out_dir, stripped_fn)
    return stripped_fn + ".full.compressed2.bed.full.entropy.bed"

def when_peak_recovered_beds(df):

    """Returns a DF annotated with peaks recovered at each round of downsampling.  Also returns those peaks as list of bedtools.  
    
    row is fully_downsampled_data row
    rep is a string either "rep1 or "rep2"
    """
        
    #Setup empty objects to append to
    peaks = {}
    all_peaks = df.filtered_moderate 
    total_bed = pybedtools.BedTool(total_bedtools.ix[df.ix[0].name[:-1]].entropy_tools)
    interval = total_bed[0]
    interval.start = 0
    interval.stop = 1
    total_peaks_detected = pybedtools.BedTool([interval])
    
    #Assume sorted order of fractions
    for name, fraction in all_peaks.iteritems():

        fraction_bed = pybedtools.BedTool(fraction)
        
        #Get the peaks that exist total that also exist in this fraction
        fraction_peaks_bed = total_bed.intersect(fraction_bed, u=True, s=True, sorted=True)
        
        #Make sure peaks haven't been detected in previous fractions
        fraction_peaks_bed = fraction_peaks_bed.intersect(total_peaks_detected, v=True, s=True, sorted=True)
            
        fraction_peaks_bed = fraction_peaks_bed.saveas("{}.{}.peaks.bed".format(total_bed.fn, os.path.basename(fraction)))
        if len(fraction_peaks_bed) != 0:
            total_peaks_detected = total_peaks_detected.cat(fraction_peaks_bed, postmerge=False).sort()

        peaks[name] = fraction_peaks_bed.fn

    return pd.Series(peaks)


# Calculate family mapped number reads as well

In [ ]:
#Get all the counts
counts_list = pd.read_csv("downsample_counts_full_v4.csv", dtype={"RBP_ID": str, "bio_rep": str, "tech_rep": str})
counts_list = counts_list.set_index(["RBP_ID", "RBP", "bio_rep", "tech_rep", "fraction"])
counts_list = counts_list.xs(1.0, level="fraction").groupby(level=['RBP_ID', "RBP", "bio_rep"]).sum()
counts_list.index.rename(["uID", "RBP", "bio_rep"], inplace=True)
counts_list['rep'] = ["rep1" if rep == "01" else "rep2" for rep in counts_list.index.get_level_values(level="bio_rep")]
counts_list.index = counts_list.index.droplevel("bio_rep")
counts_list = counts_list.set_index("rep", append=True)

In [ ]:
def family_map_read_num(read_num):
    x = np.arange(.1, 1.1, .1)
    series = pd.Series(x * read_num, index=x)
    series.index.name = "fraction"
    return series

In [ ]:
family_map_num = counts_list.unique.apply(family_map_read_num).stack()
family_map_num.index.name = "family_reads"
family_map_num.index = pd.MultiIndex.from_tuples([(x[0], x[1], x[2], str(x[3])) for x in family_map_num.index], 
                                                 names=family_map_num.index.names)


In [ ]:
merged_data = merged_data.reset_index(['Cell line'])

merged_data.index = pd.MultiIndex.from_tuples([(x[0], x[1], x[2], str(x[3])) for x in merged_data.index], 
                                                 names=merged_data.index.names)

merged_data['family_reads'] = family_map_num

merged_data.index = pd.MultiIndex.from_tuples([(x[0], x[1], x[2], float(x[3])) for x in merged_data.index], 
                                                 names=merged_data.index.names)

merged_data = merged_data.set_index(["Cell line"], append=True)
merged_data = merged_data.reorder_levels(['uID','RBP', 'Cell line','rep', "fraction"])

In [ ]:
total_bedtools = merged_data.xs(1.0, level="fraction")


In [ ]:
merged_data['reads'] = merged_data.CLIP.apply(encode_helpers.get_mapped_reads)
merged_data['fraction_overlap'] = merged_data.apply(fraction_bound, axis=1)
merged_data['fraction_not_in_total'] = merged_data.apply(fraction_not_in_total, axis=1)

In [ ]:
annotated_bedtool_header = ['chrom', 'start', "stop", "name", "score", "strand", "annotation", "gene_id"]
result = []
for name, row in tqdm(list(total_bedtools.iterrows())):
    full_file = get_entropy_from_annotated(row.input_norm)
    out_file = os.path.splitext(full_file)[0] + ".significant.bed"
    
    if os.path.exists(out_file):
        result.append(out_file)
        continue
        
    entropy = pd.read_table(full_file)
    peaks = pybedtools.BedTool(row.filtered_moderate).to_dataframe(names=annotated_bedtool_header)
    merged_peaks = pd.merge(peaks, entropy, 
                            left_on=['chrom', 'start', 'stop'], right_on=['chrom', 'start', 'stop'], how="left")
    merged_peaks = merged_peaks.sort_values(["chrom", "start", "stop"])
    if len(merged_peaks[merged_peaks.entropy.isnull()]) != 0:
        print row.input_norm
        
    merged_peaks.to_csv(out_file, sep="\t", header=False, index=False)
    result.append(out_file)
total_bedtools['entropy_tools'] = result

In [ ]:
total_datasets = len(merged_data.groupby(level=0))
num_cols = 4
num_rows = (total_datasets / 4) + 1
count = 0

with dataviz.Figure(os.path.join(img_dir, "downsampling_peaks.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    for uID, df in merged_data.groupby(level='uID'):
        count += 1
        ax = fig.add_subplot(num_rows, num_cols, count)

        for rep, dataset_df in df.groupby(level='rep'):
            rbp_name = dataset_df.index.get_level_values(level="RBP")[0]
            name = "{} {}".format(uID, rbp_name)
            ax.plot(dataset_df[reads], dataset_df.fraction_overlap, 
                    label=name, 
                    alpha=.7, 
                    linewidth=3)
            ax.xaxis.set_major_formatter(xfmt)
            ax.set_xlabel("Reads (M)")
            ax.set_ylabel("Fraction Overlapped")
            ax.set_ylim(0,1)
            sns.despine(ax=ax)
        ax.legend(fontsize=18)

In [ ]:
total_datasets = len(merged_data.groupby(level=0))
num_cols = 4
num_rows = (total_datasets / 4) + 1
count = 0

with dataviz.Figure(os.path.join(img_dir, "downsampling_peaks.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    for uID, df in merged_data.groupby(level='uID'):
        count += 1
        ax = fig.add_subplot(num_rows, num_cols, count)

        for rep, dataset_df in df.groupby(level='rep'):
            rbp_name = dataset_df.index.get_level_values(level="RBP")[0]
            name = "{} {}".format(uID, rbp_name)
            ax.plot(dataset_df[reads], dataset_df.fraction_not_in_total, 
                    label=name, 
                    alpha=.7, 
                    linewidth=3)
            ax.xaxis.set_major_formatter(xfmt)
            ax.set_xlabel("Reads (M)")
            ax.set_ylabel("Fraction Overlapped")
            ax.set_ylim(0,1)
            sns.despine(ax=ax)
        ax.legend(fontsize=18)

In [ ]:
foo = merged_data.xs(.9, level="fraction")
foo[foo.fraction_overlap < .80]

In [ ]:
#rbps_to_plot = ["204", "205", "249", "241", "223"]
rbps_to_plot = ["204", "242", "289"]

rbps_to_plot_df = merged_data.loc[rbps_to_plot]
rbps_to_plot_df = rbps_to_plot_df.xs("rep1", level="rep")

total_datasets = len(rbps_to_plot_df.groupby(level=0))
num_cols = 1
num_rows = 1
count = 0

with dataviz.Figure(os.path.join(img_dir, "downsample_figure_fraction_presentation.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    for uID, df in rbps_to_plot_df.groupby(level=0):
        rbp_name = df.index.get_level_values(level="RBP")[0]

        name = "{}".format(rbp_name)
        ax.plot(df.index.get_level_values("fraction"), df.fraction_overlap, 
                label=name, 
                alpha=.7, 
                linewidth=5)
        ax.set_xlabel("Fraction Downsampled", fontsize=20)
        ax.set_ylabel("Fraction Peaks", fontsize=20)
        ax.set_ylim(0,1)
        ax.set_title("Fraction of\nTotal Peaks Detected", fontsize=20)
        sns.despine(ax=ax)
        ax.set_xticks(np.arange(0, 1.1, .2))
        [tick.set_fontsize(16) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(16) for tick in ax.get_yticklabels()]
        #ax.axhline(.90)
        ax.axhline(.90, linestyle="--", linewidth=5, alpha=.7, color=".7")
    ax.legend(fontsize=20)
    
with dataviz.Figure(os.path.join(img_dir, "downsample_figure_presentation.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    for uID, df in rbps_to_plot_df.groupby(level=0):
        rbp_name = df.index.get_level_values(level="RBP")[0]

        name = "{}".format(rbp_name)
        ax.plot(df[reads], df.fraction_overlap, 
                label=name, 
                alpha=.7, 
                linewidth=5)
        ax.xaxis.set_major_formatter(xfmt)
        ax.set_xlabel("Reads (M)", fontsize=20)
        ax.set_ylabel("Fraction Peaks", fontsize=20)
        ax.set_ylim(0,1)
        ax.set_title("Fraction of\nTotal Peaks Recovered", fontsize=20)
        sns.despine(ax=ax)
        [tick.set_fontsize(18) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(18) for tick in ax.get_yticklabels()]

    ax.legend(fontsize=20)
    
with dataviz.Figure(os.path.join(img_dir, "downsample_figure_pub.svg"), figsize=(2.5* num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    for uID, df in rbps_to_plot_df.groupby(level=0):
        rbp_name = df.index.get_level_values(level="RBP")[0]

        name = "{}".format(rbp_name)
        ax.plot(df[reads], df.fraction_overlap, 
                label=name, 
                alpha=.7, 
                linewidth=3)
        ax.xaxis.set_major_formatter(xfmt)
        ax.set_xlabel("Reads (M)", fontsize=8)
        ax.set_ylabel("Fraction Peaks", fontsize=8)
        ax.set_ylim(0,1)
        ax.set_title("Fraction of Total Peaks Recovered", fontsize=8)
        sns.despine(ax=ax)
        [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(8) for tick in ax.get_yticklabels()]

    ax.legend(fontsize=8)

# Calculate Entropy per recovered peak

In [ ]:
# merged_data.index = merged_data.index.rename(['uID', 'Cell line', 'RBP', 'rep', 'fraction'])
# merged_data = merged_data.swaplevel("Cell line", "RBP")

In [ ]:
result = []
for name, df in tqdm(list(merged_data.groupby(level=["uID", "RBP", "Cell line", "rep"]))):
    result.append(when_peak_recovered_beds(df))
    
appearing_peaks = pd.concat(result)
appearing_peaks = appearing_peaks.rename("appearing_peaks")
merged_data = pd.concat([merged_data, appearing_peaks], axis=1)

In [ ]:
entropy_header = ["full_name", "ip_reads", "input_reads", "p_val", "fold_change", "something", "enrichment", "log10_p_val", "log2_fold_change", 'entropy']
entropy_header = annotated_bedtool_header + entropy_header

def get_total_entropy(fn):
    df = pd.read_table(fn, names=entropy_header, 
              header=None).entropy.sum()
    return df

merged_data['entropy'] = merged_data.appearing_peaks.progress_apply(get_total_entropy)

In [ ]:
def group_fraction(series):
    return  np.cumsum(series / series.sum())

merged_data['fraction_entropy'] = merged_data.entropy.groupby(level=["uID", "RBP", "Cell line", "rep"]).transform(group_fraction)

In [ ]:
total_datasets = len(merged_data.groupby(level=["uID", "RBP", "Cell line"]))
num_cols = 4
num_rows = (total_datasets / 4) + 1
count = 0

with dataviz.Figure(os.path.join(img_dir, "entropy_curve.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    for name, df in merged_data.groupby(level=["uID", "RBP", "Cell line"]):
        count += 1
        ax = fig.add_subplot(num_rows, num_cols, count)

        for rep, dataset_df in df.groupby(level="rep"):
            legend_name = "{} {}".format(uID, name[1])
            ax.plot(dataset_df[reads], dataset_df.fraction_entropy, 
                    label=legend_name, 
                    alpha=.7, 
                    linewidth=3)
            ax.xaxis.set_major_formatter(xfmt)
            ax.set_xlabel("Reads (M)")
            ax.set_ylabel("Fraction Observed Entropy")
            ax.set_ylim(0,1)
            sns.despine(ax=ax)
        ax.legend(fontsize=18)

In [ ]:
#rbps_to_plot = ["204", "205", "331", "242", "240"]
rbps_to_plot = ["204", "242", "289"]

rbps_to_plot_df = merged_data.loc[rbps_to_plot]
rbps_to_plot_df = rbps_to_plot_df.xs("rep1", level="rep")

total_datasets = len(rbps_to_plot_df.groupby(level=0))
num_cols = 1
num_rows = 1
count = 0

with dataviz.Figure(os.path.join(img_dir, "downsample_figure_entropy_pub_fraction_presentation.svg"), figsize=(5* num_cols, 5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    for name, df in rbps_to_plot_df.groupby(level=["uID", "RBP", "Cell line"]):
        legend_name = "{}".format(name[1])
        ax.plot(df.index.get_level_values("fraction"), df.fraction_entropy, 
                label=legend_name, 
                alpha=.7, 
                linewidth=5)
        ax.set_xlabel("Fraction Downsampled", fontsize=20)
        ax.set_ylabel("Fraction Entropy", fontsize=20)
        ax.set_ylim(0,1)
        ax.set_title("Fraction of Entropy Recovered", fontsize=20)
        sns.despine(ax=ax)
        [tick.set_fontsize(18) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(18) for tick in ax.get_yticklabels()]

    ax.legend(fontsize=20)
    
with dataviz.Figure(os.path.join(img_dir, "downsample_figure_entropy_pub_presentation.svg"), figsize=(5* num_cols, 5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    for name, df in rbps_to_plot_df.groupby(level=["uID", "RBP", "Cell line"]):
        legend_name = "{}".format(name[1])
        ax.plot(df[reads], df.fraction_entropy, 
                label=legend_name, 
                alpha=.7, 
                linewidth=5)
        ax.xaxis.set_major_formatter(xfmt)
        ax.set_xlabel("Reads (M)", fontsize=20)
        ax.set_ylabel("Fraction Entropy", fontsize=20)
        ax.set_ylim(0,1)
        ax.set_title("Fraction of Entropy Recovered", fontsize=20)
        sns.despine(ax=ax)
        [tick.set_fontsize(18) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(18) for tick in ax.get_yticklabels()]

    ax.legend(fontsize=20)
    
with dataviz.Figure(os.path.join(img_dir, "downsample_figure_entropy_pub.svg"), figsize=(2.5* num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    for name, df in rbps_to_plot_df.groupby(level=["uID", "RBP", "Cell line"]):
        legend_name = "{}".format(name[1])
        ax.plot(df[reads], df.fraction_entropy, 
                label=legend_name, 
                alpha=.7, 
                linewidth=3)
        ax.xaxis.set_major_formatter(xfmt)
        ax.set_xlabel("Reads (M)", fontsize=8)
        ax.set_ylabel("Fraction Entropy", fontsize=8)
        ax.set_ylim(0,1)
        ax.set_title("Fraction of Entropy Recovered", fontsize=8)
        sns.despine(ax=ax)
        [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(8) for tick in ax.get_yticklabels()]

    ax.legend(fontsize=8)
    

# How many RBPs recover by fraction, and sequencing depth

In [ ]:
x = (12 / 22.) * 2

In [ ]:
y = (6 / 22.) * 1

In [ ]:
z= (4 / 22.) * 2

In [ ]:
x+y+z

In [ ]:
fraction_above_cutoff = merged_data[merged_data.fraction_overlap > .90]
fraction_above_cutoff['fraction'] = fraction_above_cutoff.index.get_level_values("fraction")
fraction_above_cutoff = fraction_above_cutoff.groupby("fraction").count().fraction_overlap
fraction_above_cutoff = fraction_above_cutoff / fraction_above_cutoff.loc[1.0]

with dataviz.Figure(os.path.join(img_dir, "fraction_peaks_recovered_presentation.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.plot(fraction_above_cutoff.index, fraction_above_cutoff.values, linewidth=5, alpha=.7)
    ax.set_xlabel("Fraction of Reads Downsampled", fontsize=20)
    ax.set_ylabel("Fraction RBPs", fontsize=20)
    ax.set_title("Fraction of RBPs with\n>90% of peaks recovered", fontsize=20)
    sns.despine(ax=ax)
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    [tick.set_fontsize(18) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(18) for tick in ax.get_yticklabels()]
    
with dataviz.Figure(os.path.join(img_dir, "fraction_peaks_recovered.svg"), figsize=(2.5* num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.plot(fraction_above_cutoff.index, fraction_above_cutoff.values, linewidth=3, alpha=.7)
    ax.set_xlabel("Fraction of Reads Downsampled", fontsize=8)
    ax.set_ylabel("Fraction RBPs", fontsize=8)
    ax.set_title("Fraction of RBPs with\n>90% of peaks recovered", fontsize=8)
    sns.despine(ax=ax)
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]


In [ ]:
#Lets do this by read depth as well
read_increments = 10000
reads_above_cutoff = merged_data[merged_data.fraction_overlap > .90]

#I try to group reads and count at what read depth I recover more than 90% of peaks
reads_above_cutoff['fraction_reads'] = np.floor(reads_above_cutoff[reads].astype(int) / read_increments)
reads_above_cutoff = reads_above_cutoff.groupby(level=[0,1]).first()
max_reads = int(np.max(reads_above_cutoff.fraction_reads))
reads_above_cutoff_fraction = np.zeros(max_reads)

for value in reads_above_cutoff.fraction_reads:
    reads_above_cutoff_fraction[int(value):] += 1
reads_above_cutoff_fraction = reads_above_cutoff_fraction / float(reads_above_cutoff_fraction[-1])


with dataviz.Figure(os.path.join(img_dir, "read_number_peaks_recovered.svg"), figsize=(2.5* num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.plot(np.arange(0, len(reads_above_cutoff_fraction) * read_increments, read_increments), reads_above_cutoff_fraction, linewidth=3, alpha=.7)
    ax.set_xlabel("Reads (M)", fontsize=8)
    ax.set_ylabel("Fraction RBPs", fontsize=8)
    ax.set_title("Fraction of RBPs with\n>90% of peaks recovered", fontsize=8)
    ax.xaxis.set_major_formatter(xfmt)
    sns.despine(ax=ax)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]


# Do the above except with Entropy

In [ ]:
fraction_above_cutoff = merged_data[merged_data.fraction_entropy > .90]
fraction_above_cutoff = fraction_above_cutoff.groupby(level="fraction").count().fraction_entropy
fraction_above_cutoff = fraction_above_cutoff / fraction_above_cutoff.loc[1.0]
fraction_above_cutoff = fraction_above_cutoff.sort_index()

with dataviz.Figure(os.path.join(img_dir, "fraction_entropy_recovered_presentation.svg"), figsize=(3.5* num_cols, 3.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.plot(fraction_above_cutoff.index, fraction_above_cutoff.values, linewidth=3, alpha=.7)
    ax.set_xlabel("Fraction\nreads downsampled", fontsize=20)
    ax.set_ylabel("Fraction RBPs", fontsize=20)
    ax.set_title("Fraction RBPs with >90%\nentropy recovered", fontsize=20)
    sns.despine(ax=ax)
    ax.set_xticks(np.arange(0,1.2, .3))
    ax.set_yticks(np.arange(0,1.2, .2))

    [tick.set_fontsize(20) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(20) for tick in ax.get_yticklabels()]
    
with dataviz.Figure(os.path.join(img_dir, "fraction_entropy_recovered.svg"), figsize=(2.5* num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.plot(fraction_above_cutoff.index, fraction_above_cutoff.values, linewidth=3, alpha=.7)
    ax.set_xlabel("Fraction of reads downsampled", fontsize=8)
    ax.set_ylabel("Fraction RBPs", fontsize=8)
    ax.set_title("Fraction of RBPs with\n>90% entropy recovered", fontsize=8)
    sns.despine(ax=ax)
    
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]


In [ ]:
np.where(reads_above_cutoff_fraction > .80)[0][0] * read_increments

In [ ]:
#Lets do this by read depth as well
read_increments = 10000
reads_above_cutoff = merged_data[merged_data.fraction_entropy > .90]

#I try to group reads and count at what read depth I recover more than 90% of peaks
reads_above_cutoff['fraction_reads'] = np.floor(reads_above_cutoff[reads].astype(int) / read_increments)
reads_above_cutoff = reads_above_cutoff.groupby(level=[0,1]).first()
max_reads = int(np.max(reads_above_cutoff.fraction_reads))
reads_above_cutoff_fraction = np.zeros(max_reads)

for value in reads_above_cutoff.fraction_reads:
    reads_above_cutoff_fraction[int(value):] += 1
reads_above_cutoff_fraction = reads_above_cutoff_fraction / float(reads_above_cutoff_fraction[-1])

with dataviz.Figure(os.path.join(img_dir, "read_number_entropy_recovered_presentation.svg"), figsize=(3.5* num_cols, 3.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.plot(np.arange(0, len(reads_above_cutoff_fraction) * read_increments, 
                      read_increments), reads_above_cutoff_fraction, linewidth=5, alpha=.7)
    ax.set_xlabel("Reads (M)", fontsize=20)
    ax.set_ylabel("Fraction RBPs", fontsize=20)
    ax.set_title("Fraction RBPs with >90%\n of entropy recovered", fontsize=20)
    ax.axvline(3000000, linewidth=5, color=".7", linestyle="--")
    ax.xaxis.set_major_formatter(xfmt_int)
    sns.despine(ax=ax)
    #ax.set_xticks(np.arange(0,1.2, .3))
    ax.set_yticks(np.arange(0,1.2, .2))

    [tick.set_fontsize(20) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(20) for tick in ax.get_yticklabels()]
    
with dataviz.Figure(os.path.join(img_dir, "read_number_entropy_recovered.svg"), figsize=(2.5* num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.plot(np.arange(0, len(reads_above_cutoff_fraction) * read_increments, read_increments), reads_above_cutoff_fraction, linewidth=3, alpha=.7)
    ax.set_xlabel("Reads (M)", fontsize=8)
    ax.set_ylabel("Fraction RBPs", fontsize=8)
    ax.set_title("Fraction of RBPs with\n>90% of entropy recovered", fontsize=8)
    ax.axvline(3000000, linewidth=3, color=".7", linestyle="--")
    ax.xaxis.set_major_formatter(xfmt_int)
    sns.despine(ax=ax)
    
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]


In [ ]:
#Lets do this by read depth as well
read_increments = 10000
reads_above_cutoff = initial_counts[initial_counts.fraction_entropy > .90]
reads_above_cutoff = reads_above_cutoff.drop(1.0, level="fraction")

#I try to group reads and count at what read depth I recover more than 90% of peaks
reads_above_cutoff['fraction_reads'] = np.floor(reads_above_cutoff[reads].astype(int) / read_increments)
reads_above_cutoff = reads_above_cutoff.groupby(level=[0,1]).first()
max_reads = int(np.max(reads_above_cutoff.fraction_reads))

In [ ]:
with dataviz.Figure(os.path.join(img_dir, "ect_vs_saturating_read_numbers.svg"), figsize=(2.5* num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
    ax.scatter(reads_above_cutoff.eCT, reads_above_cutoff[reads], linewidth=3, alpha=.7, s=2)
    ax.set_xlabel("Reads (M)", fontsize=8)
    ax.set_ylabel("Fraction RBPs", fontsize=8)
    ax.set_title("Fraction of RBPs with\n>90% of entropy recovered", fontsize=8)
    ax.yaxis.set_major_formatter(xfmt)
    sns.despine(ax=ax)
    ax.set_ylim(0)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]


These these are the datasets that saturate before the final downsample.  What I see here is very little corrleation between eCT and number of reads it takes saturate.  This is a good thing, it means saturation is independent of eCT, so we can make our model indepenent of eCT.  I don't think I'll even write about about it because its not the case.


# RBPs don't overlap super well between bioloigcal replicates 

In [ ]:
gene_df = pd.read_table("/home/gpratt/clipper/clipper/data/regions/hg19_HepG2_genes.bed", 
                        names=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
gene_df['gene_length'] = gene_df.stop - gene_df.start
gene_lengths = gene_df.groupby("chrom").sum().gene_length
gene_lengths.to_csv("hg19_HepG2_gene_lengths.txt", sep="\t")

gene_df = pd.read_table("/home/gpratt/clipper/clipper/data/regions/hg19_K562_genes.bed", 
                        names=['chrom', 'start', 'stop', 'name', 'score', 'strand'])
gene_df['gene_length'] = gene_df.stop - gene_df.start
gene_lengths = gene_df.groupby("chrom").sum().gene_length
gene_lengths.to_csv("hg19_K562_gene_lengths.txt", sep="\t")

In [ ]:
def overlap_between_reps(row):
    rep1 = pybedtools.BedTool(row[('filtered_moderate', 'rep1')])
    rep2 = pybedtools.BedTool(row[('filtered_moderate', 'rep2')])

    return len(rep1.intersect(rep2, u=True, s=True)) / float(len(rep1))

def fisher_between_reps(row):
    rep1 = pybedtools.BedTool(row[('filtered_moderate', 'rep1')])
    rep2 = pybedtools.BedTool(row[('filtered_moderate', 'rep2')])

    if row.name[1] == "HepG2":
        species = "hg19_HepG2_gene_lengths.txt"
    elif row.name[1] == "K562":
        species = "hg19_K562_gene_lengths.txt"
    species = "hg19_gene_length_test.txt"
    return rep1.fisher(rep2, g=species)

root_dir = "/home/gpratt/projects/encode/analysis/peak_reanalysis_v14/assigned/"
def shuffle_between_reps(row):
    rep1 = row[('filtered_moderate', 'rep1')]
    rep2 = row[('filtered_moderate', 'rep2')]

    overlap = []
    for x in range(3):
        rand_1 = os.path.join(root_dir, os.path.basename(rep1) + ".clip_formatted.bed.all.rand.{}.BED".format(str(x)))
        rand_2 = os.path.join(root_dir, os.path.basename(rep2) + ".clip_formatted.bed.all.rand.{}.BED".format(str(x)))
        
        rand_1 = pybedtools.BedTool(rand_1)
        rand_2 = pybedtools.BedTool(rand_2)

        overlap.append(len(rand_1.intersect(rand_2, u=True, s=True)) / float(len(rand_1)))
    return overlap

In [ ]:
rep_comparisons = merged_data.xs(1.0, level="fraction").unstack()
rep_comparisons['fraction_overlap'] = rep_comparisons.apply(overlap_between_reps, axis=1)

rep_comparisons['fisher'] = rep_comparisons.apply(fisher_between_reps, axis=1)
rep_comparisons['fisher_two_tail_pvalues'] = rep_comparisons['fisher'].apply(lambda x: x.two_tail)

#this is broken at the moment
# rep_comparisons['shuffle'] = rep_comparisons.apply(shuffle_between_reps, axis=1)
# rep_comparisons['shuffle_mean'] = rep_comparisons['shuffle'].apply(lambda x: np.mean(x))

In [ ]:
with dataviz.Figure(os.path.join(img_dir, "peak_overlap.svg"), figsize=(4* 1, 4*1)) as fig:
    ax = fig.add_subplot(1,1,1)
    sns.violinplot(rep_comparisons["fraction_overlap", "rep1"], cut=True, ax=ax, orient='v')
    sns.despine(ax=ax)
    ax.set_xlabel("RBPs")
    ax.set_ylabel("Fraction overlapping between technical replicates")
    ax.set_title("Peak Overlap Between\nBiological Replicates")

# Test question about shitty peaks
1. Of the peaks that overlap how much does the fold change, change, how much does the p-value change?
2. Of the peaks that don't overlap and are found by deeper sequencing, there are two options,  Are we finding shittier peaks? Or are we finding peaks in more lowly expressed Genes?  

Addressing fold change question

This data isn't interesting, commenting until it becomes important enough to update with new layout

In [ ]:
# def intersect_to_df(tool_1, tool_2):
#     intersected_bed = tool_1.intersect(tool_2, s=True, wo=True)

#     intersected_df = intersected_bed.to_dataframe(names=['fraction_chrom', 'fraction_start', 
#                                         'fraction_stop', 'fraction_pval', 
#                                         'fraction_log2fc', 'fraction_strand',
#                                        'full_chrom', 'full_start', 
#                                         'full_stop', 'full_pval', 
#                                         'full_log2fc', 'full_strand',
#                                        'full_annotation', 'full_gene', 'foo'])
#     intersected_df = intersected_df.groupby(['fraction_chrom', 'fraction_start', 'fraction_stop']).first()
#     return intersected_df

In [ ]:
# total_bed = pybedtools.BedTool(merged_data.ix[2].rep1_filtered_moderate)
# fraction_bed = pybedtools.BedTool(make_and_filter_clipper_moderate(fully_downsampled_data.ix[2].CLIP_rep1_01))
# intersected_df = intersect_to_df(fraction_bed, total_bed)


In [ ]:
# len(total_bed), len(fraction_bed), len(intersected_df)

In [ ]:
# num_cols = 1
# num_rows = 2
# with dataviz.Figure(os.path.join(img_dir, "peak_overlap.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
#     ax = fig.add_subplot(num_cols,num_rows,1)
    
    
#     ax.scatter(intersected_df.fraction_log2fc, intersected_df.full_log2fc, alpha=.7, color='.2')
    
#     linregress = scipy.stats.linregress(intersected_df.fraction_log2fc, intersected_df.full_log2fc)
#     fit = np.poly1d([linregress.slope, linregress.intercept])
#     ax.plot([min(intersected_df.fraction_log2fc), max(intersected_df.fraction_log2fc)],
#             [fit(min(intersected_df.fraction_log2fc)), fit(max(intersected_df.fraction_log2fc))],
#            linewidth=3, alpha=.7)
    
#     sns.despine(ax=ax)
#     ax.set_xlabel("Fraction log2 fc")
#     ax.set_ylabel("Total log2 fc")
#     ax.set_xlim(2,10)
#     ax.set_ylim(2,10)
    
#     ax = fig.add_subplot(num_cols,num_rows,2)
#     ax.scatter(intersected_df.fraction_pval, intersected_df.full_pval, alpha=.7, color='.2')
    
#     linregress = scipy.stats.linregress(intersected_df.fraction_pval, intersected_df.full_pval)
#     fit = np.poly1d([linregress.slope, linregress.intercept])
#     ax.plot([min(intersected_df.fraction_pval), max(intersected_df.fraction_pval)],
#             [fit(min(intersected_df.fraction_pval)), fit(max(intersected_df.fraction_pval))],
#             linewidth=3, alpha=.7)
    
#     sns.despine(ax=ax)
#     ax.set_xlabel("Fraction pval")
#     ax.set_ylabel("Total pval")
#     ax.set_xlim(0,400)
#     ax.set_ylim(0,400)

This isn't a really good way to look at the data because it starts digging into issues with clipper.  Thats not the question we are asking.  We want to know what do the undiscovered peaks look like, not why were they undiscovered.  

In [ ]:
# print scipy.stats.linregress(intersected_df.fraction_pval, intersected_df.full_pval)
# print scipy.stats.linregress(intersected_df.fraction_log2fc, intersected_df.full_log2fc)

Need to explain why the slope is less than 1

So this is telling me that my p-value and fold change isn't really changing.  Do I want to ask how many are lost because they are detected, but not significant, and how many are lost because they aren't detected at all.  

In [ ]:
# total_bed = pybedtools.BedTool(fully_downsampled_data.ix[1].rep1_filtered_moderate)
# fraction_bed = pybedtools.BedTool(fully_downsampled_data.ix[1].CLIP_rep1_01)

# unfiltered = intersect_to_df(fraction_bed, total_bed)
# no_pass = unfiltered[~unfiltered.index.isin(intersected_df.index)]
# (no_pass.fraction_pval > 3) & (no_pass.fraction_log2fc > 3)

# print len(no_pass), sum(no_pass.fraction_log2fc > 3)

Of the peaks not called significant all but two of them are not called significant due to a low p-value.  Not a low fold change

I need to change approaches and look at peaks that weren't identified, agnostic of how they weren't identified, then run clip analysis on the result

# Of the peaks that I don't identify quickly, what is the quality of motif recovery?

This is an attempt to run homer, that I don't like / need anymore

In [ ]:
# total_bed = pybedtools.BedTool(fully_downsampled_data.ix[1].rep1_filtered_moderate)
# fraction_bed = pybedtools.BedTool(make_and_filter_clipper_moderate(fully_downsampled_data.ix[1].CLIP_rep1_01))

# Of the peaks that aren't overlapping, what is the expression level of the RNA thats being detected

In [ ]:
# #K562 Expression values
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF286GLL/@@download/ENCFF286GLL.tsv", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF286GLL.tsv")
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF986DBN/@@download/ENCFF986DBN.tsv", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF986DBN.tsv")

# #HepG2 Expression values
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF533XPJ/@@download/ENCFF533XPJ.tsv", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF533XPJ.tsv")
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF321JIT/@@download/ENCFF321JIT.tsv", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF321JIT.tsv")

# #K562 bams
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF131IST/@@download/ENCFF131IST.bam", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF131IST.bam")
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF726SMY/@@download/ENCFF726SMY.bam", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF726SMY.bam")

# #HepG2 bams
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF770XVY/@@download/ENCFF770XVY.bam", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF770XVY.bam")
# foo = urllib.urlretrieve("https://www.encodeproject.org/files/ENCFF002PXG/@@download/ENCFF002PXG.bam", "/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF002PXG.bam")


In [ ]:
k562_rep1 = pd.read_table("/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF286GLL.tsv")
k562_rep2 = pd.read_table("/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF986DBN.tsv")

hepg2_rep1 = pd.read_table("/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF533XPJ.tsv")
hepg2_rep2 = pd.read_table("/projects/ps-yeolab3/encode/analysis/rnaseq_bams_v2/ENCFF321JIT.tsv")



In [ ]:
df_columns = [u'chrom', u'start', u'end', u'name', u'score', u'strand', u'annotation',
       u'gene_id', u'transcript_id(s)', u'length', u'effective_length',
       u'expected_count', u'TPM', u'FPKM', u'posterior_mean_count',
       u'posterior_standard_deviation_of_count', u'pme_TPM', u'pme_FPKM',
       u'TPM_ci_lower_bound', u'TPM_ci_upper_bound', u'FPKM_ci_lower_bound',
       u'FPKM_ci_upper_bound', u'overlap_type']

In [ ]:
def calculate_ntile(df, ntile=4):
    "Takes a dataframe and returns a multi-index dataframe where the first index is now the quartile"
    ntiles = np.arange(0, len(df) + 1, len(df) / ntile)

    results = {}
    for x, (start, finish) in enumerate(zip(ntiles, ntiles[1:])):
        results[x + 1] = df.iloc[start:finish]

    quartiled = pd.concat(results, names=['quartile', 'id'])
    quartiled['quartile'] = quartiled.index.get_level_values(level=0)
    return quartiled


hepg2_rep1_tpm_sorted = hepg2_rep1.sort_values(by="TPM", axis=0)
hepg2_rep1_tpm_sorted = hepg2_rep1_tpm_sorted[hepg2_rep1_tpm_sorted.TPM > 0]
hepg2_rep1_tpm_sorted = calculate_ntile(hepg2_rep1_tpm_sorted)

k562_rep1_tpm_sorted = k562_rep1.sort_values(by="TPM", axis=0)
k562_rep1_tpm_sorted = k562_rep1_tpm_sorted[k562_rep1_tpm_sorted.TPM > 0]
k562_rep1_tpm_sorted = calculate_ntile(k562_rep1_tpm_sorted)


In [ ]:
def get_recovered_peaks(df):
    cell_line = df.index.get_level_values(level="Cell line")[0]
    if cell_line == "HepG2":
        cell_expression = hepg2_rep1_tpm_sorted
    elif cell_line == "K562":
        cell_expression = k562_rep1_tpm_sorted
    else:
        print "error"
    
    result = {}
    for name, row in df.iterrows():
        result[name] = pd.read_table(row.appearing_peaks, names=entropy_header)
        result[name]['reads'] = row.reads
    result = pd.concat(result)
    result.index.names = ['uID', "RBP", "Cell line", "rep", "fraction", "peak"]

    result = result.reset_index()
    peaks_and_expression = pd.merge(result, cell_expression, 
         left_on='gene_id',
         right_on='gene_id', how="left")
    return peaks_and_expression

def plot_peak_expression_differences(df_rep1, df_rep2, out_file):
    """plots the expression of each gene peaks are bound to in both rep1 and rep2"""
    num_cols = 1
    num_rows = 2

    df_rep1 = df_rep1.sort_values("fraction")
    df_rep2 = df_rep2.sort_values("fraction")

    with dataviz.Figure(os.path.join(img_dir, out_file), figsize=(4* num_rows, 4*num_cols)) as fig:
        ax = fig.add_subplot(num_cols,num_rows,1)
        sns.boxplot(x="TPM", y="fraction", data=df_rep1, ax=ax, orient='h')
        ax.set(xscale="log")
        sns.despine(ax=ax)
        ax.set_title("Gene Expression of Peaks Rep1")
        ax.set_ylabel("Fraction Peaks Recovered")
        ax.set_yticklabels(np.arange(.1,1.1, .1))

        ax = fig.add_subplot(num_cols,num_rows,2)
        sns.boxplot(x="TPM", y="fraction", data=df_rep2, ax=ax, orient='h')
        ax.set(xscale="log")
        sns.despine(ax=ax)
        ax.set_title("Gene Expression of Peaks Rep 2")
        ax.set_ylabel("Fraction Peaks Recovered")
        ax.set_yticklabels(np.arange(.1,1.1, .1))

In [ ]:
grp = merged_data.groupby(level=["uID", "RBP", 'Cell line',  "rep"])
df_rep1 = grp.get_group(('204', 'RBFOX2', 'HepG2', "rep1"))
df_rep2 = grp.get_group(('204', 'RBFOX2', 'HepG2', "rep2"))


df_rep1 = get_recovered_peaks(df_rep1)
df_rep2 = get_recovered_peaks(df_rep2)

plot_peak_expression_differences(df_rep1, df_rep2, "rbfox2_peak_expression.svg")

In [ ]:
df_rep1_groups = df_rep1.groupby("fraction")
df_rep2_groups = df_rep2.groupby("fraction")

print "rep1 tests"
for group1, group2 in zip(sorted(df_rep1_groups.groups.keys())[:-1], sorted(df_rep1_groups.groups.keys())[1:]):
    print group1, group2, scipy.stats.mannwhitneyu(df_rep1_groups.get_group(group1).TPM, df_rep1_groups.get_group(group2).TPM) 

print
print "rep2 tests"

for group1, group2 in zip(sorted(df_rep2_groups.groups.keys())[:-1], sorted(df_rep2_groups.groups.keys())[1:]):
    print group1, group2, scipy.stats.mannwhitneyu(df_rep2_groups.get_group(group1).TPM, df_rep2_groups.get_group(group2).TPM) 

I've shown for RBFOX2 that gene expression doesn't matter when it comes to peak recovery.  The only stastical difference between peak quality is in the tranisition from the first to the second group.  Possibly can do all by all comparisons, but that doesn't make a bunch of sense.  

# Try a Chi-Sq Test for RBFOX2, breaking up peaks into quartiles

# Figure out this lack of gene expression pattern for the rest of the RBPs

In [ ]:
mann_whitney_pvalues = {}
chisq_pvalues = {}
for name, df in tqdm(list(merged_data.groupby(level=["uID", "RBP", "Cell line"]))): 
    uID, rbp, cell_line = name
    
    grp = df.groupby(level="rep")
    df_rep1 = get_recovered_peaks(grp.get_group("rep1"))
    df_rep2 = get_recovered_peaks(grp.get_group("rep2"))

    plot_peak_expression_differences(df_rep1, df_rep2, "{}_{}_{}.expression_differences.svg".format(uID, rbp, cell_line))
    
    df_rep1_groups = df_rep1.groupby("fraction")
    df_rep2_groups = df_rep2.groupby("fraction")

    chisq_pvalues[(uID, 'rep1')] = scipy.stats.chi2_contingency(df_rep1.groupby(["fraction", 'quartile']).count().FPKM.unstack().fillna(0))[1]
    chisq_pvalues[(uID, 'rep2')] = scipy.stats.chi2_contingency(df_rep2.groupby(["fraction", 'quartile']).count().FPKM.unstack().fillna(0))[1]

    for group1, group2 in zip(sorted(df_rep1_groups.groups.keys())[:-1], sorted(df_rep1_groups.groups.keys())[1:]):
        pvalue = scipy.stats.mannwhitneyu(df_rep1_groups.get_group(group1).TPM, df_rep1_groups.get_group(group2).TPM).pvalue
        mann_whitney_pvalues[(uID, 'rep1', group2)] = {"pvalue": pvalue}
    
    for group1, group2 in zip(sorted(df_rep2_groups.groups.keys())[:-1], sorted(df_rep2_groups.groups.keys())[1:]):
        pvalue = scipy.stats.mannwhitneyu(df_rep2_groups.get_group(group1).TPM, df_rep2_groups.get_group(group2).TPM).pvalue
        mann_whitney_pvalues[(uID, 'rep2', group2)] = {"pvalue": pvalue}
mann_whitney_pvalues = pd.DataFrame(mann_whitney_pvalues).T

In [ ]:
mann_whitney_pvalues.groupby(level=2).count()

In [ ]:
#Easier than parsing the index, just add the fractions in as their own column
mann_whitney_pvalues = mann_whitney_pvalues.reset_index()
mann_whitney_pvalues.columns = ['uID', 'rep', 'fraction', 'pvalue']

In [ ]:
#filtered_mann_whitney_pvalues = mann_whitney_pvalues[mann_whitney_pvalues.pvalue < (.05 / len(mann_whitney_pvalues))]
filtered_mann_whitney_pvalues = mann_whitney_pvalues[mann_whitney_pvalues.pvalue < (.05)]

In [ ]:
p_value_fraction = filtered_mann_whitney_pvalues.groupby("fraction").count().pvalue / mann_whitney_pvalues.groupby("fraction").count().pvalue
p_value_fraction = p_value_fraction.fillna(0)

In [ ]:
num_cols = 1
num_rows = 2
with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "mann_p_value_test.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)

    sns.barplot(x=p_value_fraction.index, y=p_value_fraction.values, ax=ax)
    sns.despine(ax=ax)
    ax.set_xlabel("Fraction Peaks Recovered")
    ax.set_ylabel("Fraction of experiments with\nsignificant change between downsampling steps")
    ax.set_title("All Experments with significant changes")

# Lets look at the Chiseq values

In [ ]:
chisq_pvalues = pd.Series(chisq_pvalues)
chisq_pvalues = pd.DataFrame(chisq_pvalues, columns=['p-value'])
chisq_pvalues['neg_p_value'] = -1 * np.log10(chisq_pvalues["p-value"])

In [ ]:
neg_05 = -1 * np.log10(.05 / len(chisq_pvalues))

In [ ]:
neg_05

In [ ]:
with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "chisq_independence_test.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    
    sns.distplot(chisq_pvalues[~np.isinf(chisq_pvalues['neg_p_value'])].neg_p_value, kde=False, ax=ax)
    sns.despine(ax=ax)
    ax.axvline(neg_05, linestyle="--", alpha=.7, color=".2")
    ax.set_xlim(0,)
    ax.set_xlabel("-$log_{10}$ P Value")
    ax.set_ylabel("Number of Experiments")
    ax.set_title("Chi-Sq Indepdendence Test")

# Look at reads per peak vs RPKM of Gene

In [ ]:
peaks = pd.read_table(total_bedtools.ix[('204', "RBFOX2", "HepG2", "rep1")]['entropy_tools'], names=entropy_header)

In [ ]:
peaks_and_expression = pd.merge(peaks, hepg2_rep1, how="left", left_on="gene_id", right_on="gene_id")
peaks_and_expression['normalized_read_count'] = peaks_and_expression.ip_reads / (peaks_and_expression.stop - peaks_and_expression.start)

#I drop NA values because sometimes a peak is annotated to bind to two genes, in this case I don't make a decision which gene I choose
peaks_and_expression = peaks_and_expression.dropna()

#Because I'm plotting on a log plot, if the TPM is 0 I can't use it.  So I set the TPM to the next lowest expression level
min_expression = peaks_and_expression[peaks_and_expression.TPM != 0].TPM.min()
peaks_and_expression.ix[peaks_and_expression.TPM == 0, "TPM"] = min_expression

In [ ]:
scipy.stats.linregress(np.log10(peaks_and_expression.normalized_read_count),
                        np.log10(peaks_and_expression.TPM)).rvalue

In [ ]:
num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join(img_dir, "peaks_expression_correlation.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(1,1,1)
    #ax.scatter(peaks_and_expression.normalized_read_count, peaks_and_expression.TPM, alpha=.7, color=".7", rasterized=True)

    sns.regplot(peaks_and_expression.TPM, 
                peaks_and_expression.normalized_read_count, 
                ci=None,
                color=".7",
                scatter_kws={"alpha":.7,},
                line_kws={"color": 'b', "alpha": .7},
                
                ax=ax)
        
    ax.set_ylabel("$log_{10}$ Reads in Peaks")
    ax.set_xlabel("$log_{10}$ TPM")
    ax.set_xlim(.01, 100000)
    ax.set_ylim(.01, 100000)
    ax.set_yscale("log", basey=10)
    ax.set_xscale("log", basex=10)
    ax.set_title("RBFOX2 Gene Expression Correlation")
    sns.despine(ax=ax)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]
    ax.legend()


In [ ]:
best_peaks_and_expression = peaks_and_expression.sort_values("normalized_read_count", ascending=False)
best_peaks_and_expression = best_peaks_and_expression.groupby("gene_id").first()

In [ ]:
linregress = scipy.stats.linregress(np.log10(best_peaks_and_expression.TPM), 
                                   np.log10(best_peaks_and_expression.normalized_read_count), 
                                  )
poly = np.poly1d([linregress.slope, linregress.intercept])

num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join(img_dir, "peaks_expression_correlation_best_peak_presentation.svg"), figsize=(5.0 * num_cols, 4.0*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
#     ax.scatter(best_peaks_and_expression.TPM,
#                best_peaks_and_expression.normalized_read_count,  
#                alpha=.7, color=".7", rasterized=True)
    
    hb = ax.hexbin(best_peaks_and_expression.TPM, best_peaks_and_expression.normalized_read_count, 
                   gridsize=25, 
                   bins='log',
                   cmap='Greys',
                   xscale='log',
                   yscale='log',
                   mincnt=.001
                  )
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label('log10(counts)', fontsize=20)

    ax.plot(np.arange(.01, 100000),
            np.power(10, poly(np.log10(np.arange(.01, 100000))))
           )

    ax.set_ylabel("Reads in Peaks", fontsize=20)
    ax.set_xlabel("TPM", fontsize=20)
    ax.set_xlim(.01, 100000)
    ax.set_ylim(.01, 100000)
    ax.set_yscale("log", basey=10)
    ax.set_xscale("log", basex=10)
    ax.set_title("RBFOX2 Expression Correlation", fontsize=20)
    
    [tick.set_fontsize(16) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(16) for tick in ax.get_yticklabels()]
    sns.despine(ax=ax)
    
num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join(img_dir, "peaks_expression_correlation_best_peak.svg"), figsize=(3.0 * num_cols, 2.5*num_rows)) as fig:
    ax = fig.add_subplot(num_rows, num_cols, 1)
#     ax.scatter(best_peaks_and_expression.TPM,
#                best_peaks_and_expression.normalized_read_count,  
#                alpha=.7, color=".7", rasterized=True)

    hb = ax.hexbin(best_peaks_and_expression.TPM, best_peaks_and_expression.normalized_read_count, 
                   gridsize=25, 
                   bins='log',
                   cmap='Greys',
                   xscale='log',
                   yscale='log',
                   mincnt=.001
                  )
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label('log10(counts)', fontsize=8)

    ax.plot(np.arange(.01, 100000),
            np.power(10, poly(np.log10(np.arange(.01, 100000))))
           )

    ax.set_ylabel("Reads in Peaks", fontsize=8)
    ax.set_xlabel("TPM", fontsize=8)
    ax.set_xlim(.01, 100000)
    ax.set_ylim(.01, 100000)
    ax.set_yscale("log", basey=10)
    ax.set_xscale("log", basex=10)
    ax.set_title("RBFOX2 Expression Correlation", fontsize=8)
    
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]
    sns.despine(ax=ax)

In [ ]:
print scipy.stats.linregress(best_peaks_and_expression.normalized_read_count,
                        best_peaks_and_expression.TPM)

print scipy.stats.linregress(np.log10(best_peaks_and_expression.normalized_read_count), 
                             best_peaks_and_expression.TPM)

So we won't detect lowly expressed genes, but if you sequence a bit deeper you'll start picking that stuff up quickly, and it will stay consistant

I want to figure out a way to show change in detection between downsampled and non-downsampled datasets, maybe I'll go dive into the litature, althought this is somewhat of a unique question.  I think I'm honestly happy with my plot / regression.  The downsampling is unnessessary to solve this question, or at least its overkill.  

In [ ]:
def get_full_from_annotated(fn):
    stripped_fn = ".".join(fn.split(".")[:-3])
    return stripped_fn + ".full.compressed2.bed.full"

In [ ]:
#Some Nans because a few genes don't have any gene expression at all.  Those get filtered out for the quartile
#calculations up above

def get_fraction_expressed(group, fraction=.1):
    """Returns the gene expression in each group, gets just that fractional expression, to describe new peaks detected"""
    group = group.sort_values()
    #return group.mean()
    return group.iloc[int(len(group) * fraction)]

def merge_peak_and_read_counts(df):
        
    df = get_recovered_peaks(df)
        
    df['stop'] = df.stop.astype(int)
    df['normalized_read_count'] = df.ip_reads / (df.stop - df.start)

    #Because I'm plotting on a log plot, if the TPM is 0 I can't use it.  So I set the TPM to the next lowest expression level
    min_expression = df[df.TPM != 0].TPM.min()
    df.ix[df.TPM == 0, "TPM"] = min_expression

    return df

In [ ]:
r_values = defaultdict(dict)
for name, df in tqdm(list(merged_data.groupby(level=["uID", "RBP", "Cell line", "rep"]))):
    cell_type = name[2]
    
    peaks_and_expression = merge_peak_and_read_counts(df).dropna()
    
    best_peaks_and_expression = peaks_and_expression.sort_values("normalized_read_count", ascending=False)
    best_peaks_and_expression = best_peaks_and_expression.groupby("gene_id").first()
    
    bottom_10_percent = peaks_and_expression.groupby('fraction').TPM.agg(get_fraction_expressed)

    r_values[name] = {"r-value": scipy.stats.linregress(np.log10(peaks_and_expression.normalized_read_count),
                                                                   np.log10(peaks_and_expression.TPM)).rvalue,
                      "r-value_best": scipy.stats.linregress(np.log10(best_peaks_and_expression.normalized_read_count),
                                                             np.log10(best_peaks_and_expression.TPM)).rvalue,
                                 'mean': bottom_10_percent.mean(),
                                 'var': bottom_10_percent.var()}
    
r_values = pd.DataFrame(r_values)
r_values = r_values.T

In [ ]:
r2_values = np.power(r_values['r-value'], 2)
print r2_values.mean()

r2_values_to_share = r2_values.unstack()
r2_values_to_share.to_csv("/projects/ps-yeolab3/encode/analysis/website_figures/peak_and_gene_expression_correlation.tsv", sep="\t")

In [ ]:
num_rows = 1
num_cols = 1

with dataviz.Figure(os.path.join(img_dir, "rpkm_read_count_correlations_presentation.svg"), figsize=(3* num_rows, 3*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    sns.violinplot(r2_values, cut=True, linewidth=.5, ax=ax)
    sns.despine(ax=ax)
    ax.set_xlabel("R2", fontsize=18)
    ax.set_ylabel("All Experiments", fontsize=18)
    ax.set_title("Correlation between reads\nper peak and TPM of gene", fontsize=18)
    ax.set_xlim(0, 1)
    [tick.set_fontsize(18) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(18) for tick in ax.get_yticklabels()]

with dataviz.Figure(os.path.join(img_dir, "rpkm_read_count_correlations.svg"), figsize=(2.5* num_rows, 2.5*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    sns.violinplot(r2_values, cut=True, linewidth=.5, ax=ax)
    sns.despine(ax=ax)
    ax.set_xlabel("R2", fontsize=8)
    ax.set_ylabel("All Experiments", fontsize=8)
    ax.set_title("Correlation between reads\nper peak and TPM of gene", fontsize=8)
    ax.set_xlim(0, 1)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]


In [ ]:
r2_values.sort_values(ascending=False)

Can explain that high correlation because its coating RBPs.  If an RBP coats is likely to have more correlated peak and gene expression

In [ ]:
r2_best_values = np.power(r_values['r-value_best'], 2)
print r2_best_values.mean()

r2_best_values_to_share = r2_best_values.unstack()
r2_best_values_to_share.to_csv("/projects/ps-yeolab3/encode/analysis/website_figures/peak_and_gene_expression_correlation_best.tsv", sep="\t")

In [ ]:
num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join(img_dir, "rpkm_read_count_correlations.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    sns.violinplot(r2_best_values, cut=True)
    sns.despine(ax=ax)
    ax.set_xlabel("$R^2$ Value")
    ax.set_ylabel("All Experiments")
    ax.set_title("Correlation between reads\nper peak in gene and TPM of gene")
    ax.set_xlim(0, 1)

# At what sequencing depth can you first detect a peak?

In [ ]:
results = []
for name, df in tqdm(list(merged_data.groupby(level=["uID", "RBP", "Cell line"]))): 
    uID, rbp, cell_line = name
    
    grp = df.groupby(level="rep")
    results.append(get_recovered_peaks(grp.get_group("rep1")))
    results.append(get_recovered_peaks(grp.get_group("rep2")))
results = pd.concat(results)

In [ ]:
first_detected = results.sort_values("reads").groupby("gene_id").first()

In [ ]:
len(first_detected)

In [ ]:
first_detected = first_detected[first_detected.TPM < 20000]

In [ ]:
num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join(img_dir, "reads_when_first_detected.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    ax.scatter(first_detected.TPM, first_detected.reads, alpha=".7", color=".7")
    sns.despine(ax=ax)
    ax.set_xlabel("TPM")
    ax.set_ylabel("Total Reads in Dataset")
    ax.set_xlim(.01, 10000)
    ax.set_ylim(10000, 16000000)
    ax.set_yscale("log", basey=10)
    ax.set_xscale("log", basex=10)


In [ ]:
mean_detected = results.sort_values("reads").groupby("gene_id").mean()

In [ ]:
print first_detected[first_detected.TPM < .1].reads.min()
print first_detected[first_detected.TPM < .1].reads.max()

In [ ]:
print first_detected[first_detected.TPM < .1].reads.median()

In [ ]:
print first_detected[first_detected.TPM < .1].reads.mean()
print first_detected[first_detected.TPM < .1].reads.var()

In [ ]:
1 * np.power(10, 6)

In [ ]:
num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join(img_dir, "reads_when_mean_detected_hexbin_presentation.svg"), figsize=(5.0* num_cols, 4.* num_rows)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    hb = ax.hexbin(first_detected.TPM, first_detected.reads, gridsize=25, 
                   bins='log',
                   cmap='Greys',
                   xscale='log',
                   yscale='log',
                   mincnt=.01)
    ax.set_title("Number of reads in dataset\nrequired to detect peak in gene", fontsize=18)
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label('log10(counts)', fontsize=18)
    ax.set_xlim(.01, 10000)
    ax.set_ylim(10000, 16000000)
    ax.set_xlabel("TPM", fontsize=18)
    ax.set_ylabel("Reads in Dataset", fontsize=18)
    [tick.set_fontsize(16) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(16) for tick in ax.get_yticklabels()]

    sns.despine(ax=ax)
    
with dataviz.Figure(os.path.join(img_dir, "reads_when_mean_detected_hexbin_paper.svg"), figsize=(2.5* num_rows, 2.5*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    hb = ax.hexbin(first_detected.TPM, first_detected.reads, gridsize=25, 
                   bins='log',
                   cmap='Greys',
                   xscale='log',
                   yscale='log',
                   mincnt=.01)
    ax.set_title("Reads to detect peak", fontsize=8)
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label('log10(counts)', fontsize=8)
    ax.set_xlim(.01, 10000)
    ax.set_ylim(10000, 16000000)
    ax.set_xlabel("TPM", fontsize=8)
    ax.set_ylabel("Reads", fontsize=8)
    [tick.set_fontsize(6) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(6) for tick in ax.get_yticklabels()]

    sns.despine(ax=ax)

In [ ]:
linregress = scipy.stats.linregress(np.log10(first_detected.TPM), np.log10(first_detected.reads))
linregress

So that fits nicely, and gives me a good model.  If you want to detect reads at say...

In [ ]:
poly = np.poly1d([linregress.slope, linregress.intercept])

In [ ]:
np.power(10, poly(np.log10(.01)))

In [ ]:
num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join(img_dir, "reads_when_mean_detected.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    ax.scatter(mean_detected.TPM, mean_detected.reads, alpha=".7", color=".7")
    sns.despine(ax=ax)
    ax.set_xlabel("TPM")
    ax.set_ylabel("Total Reads in Dataset")
    ax.set_xlim(.01, 10000)
    ax.set_ylim(10000, 16000000)
    ax.set_yscale("log", basey=10)
    ax.set_xscale("log", basex=10)


Hmm... I don't know what to do here, I could dive deeper, and figure out the number of reads I need for a given gene 

# Lets look at de-novo motif identification, and then fraction of motifs detected

In [ ]:
# results = []
# for peak in peaks:
#     results.append(format_clip_analysis(peak.fn, "hg19_HepG2", "/projects/ps-yeolab3/encode/analysis/encode_v12/204_01_RBFOX2.merged.r2.bam"))

In [ ]:
# job_name = "denovo_motif_analysis"
# job = qtools.Submitter(commands=results, 
#                  job_name="{}".format(job_name), 
#                  sh_filename="/home/gpratt/projects/idr/scripts/{}.sh".format(job_name),
#                 array=True,
#                 walltime="2:00:00",
#                 out_filename="/home/gpratt/projects/idr/scripts/{}.out".format(job_name),
#                 err_filename="/home/gpratt/projects/idr/scripts/{}.err".format(job_name),
#                 queue="home-yeo")
# job.job()

# print "/home/gpratt/projects/idr/scripts/{}.sh".format(job_name)

The denovo results are running, now to just check the actual motif presence

In [ ]:
GCATG_hg19_motif = pybedtools.BedTool("/projects/ps-yeolab3/oolite_backup/gpratt/projects/pipeline_analysis/motifs/hg19/motif_GCATG.BED").sort().saveas()
merged_data['num_peaks'] = merged_data.appearing_peaks.progress_apply(lambda x: len(pybedtools.BedTool(x)))


In [ ]:
def calculate_motif_recovery(df, motif):
    df = df.copy()
    df['overlap'] = df.appearing_peaks.apply(lambda x: len(pybedtools.BedTool(x).intersect(motif, 
                                                                                           u=True, 
                                                                                           s=True, sorted=True)))
    df['fraction_overlap_motif'] = df.overlap / df.num_peaks
    
    df['total_peaks'] = df.num_peaks.cumsum()
    df['total_overlap'] = df.overlap.cumsum()
    df['fraction_total_overlap_motif'] = df.total_overlap / df.total_peaks
    return df

def kmer_calc(fn, kmer):
    fasta_name = fn + ".fasta"
    peak = pybedtools.BedTool(fn)
    
    fasta_peaks = peak.sequence(fi="/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa", fo=fasta_name, s=True)
    result = []
    for read in SeqIO.parse(fasta_name, "fasta"):
        result.append(kmer in read.upper())
    return sum(result)

def calculate_motif_recovery_kmer(df, kmer):
    real_kmer_calc = functools.partial(kmer_calc, kmer=kmer)
    df['overlap_kmer'] = df.appearing_peaks.apply(real_kmer_calc)
    df['fraction_overlap_kmer'] = df.overlap_kmer / df.num_peaks
    return df

In [ ]:
rep1 = merged_data.ix[('204', "RBFOX2", 'HepG2', 'rep1')]
rep2 = merged_data.ix[('204', "RBFOX2", 'HepG2', 'rep2')]

motif_recovery_rep1 = calculate_motif_recovery(rep1, GCATG_hg19_motif)
motif_recovery_rep2 = calculate_motif_recovery(rep2, GCATG_hg19_motif)

In [ ]:
num_cols = 1
num_rows = 2

x = np.concatenate([motif_recovery_rep1.index.values, motif_recovery_rep2.index.values])
y = np.concatenate([motif_recovery_rep1.fraction_overlap_motif, motif_recovery_rep2.fraction_overlap_motif])

print scipy.stats.linregress(x,y)

with dataviz.Figure(os.path.join(img_dir, "fraction_of_ugcaug_recovered.svg"), figsize=(2.5* num_rows, 2.5*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    ax.plot(motif_recovery_rep1.index, motif_recovery_rep1.fraction_overlap_motif, linewidth=3, alpha=.7, label="Rep 1")
    ax.plot(motif_recovery_rep2.index, motif_recovery_rep2.fraction_overlap_motif, linewidth=3, alpha=.7, label="Rep 2")

    ax.set_ylabel("Fraction containing motif", fontsize=8)
    ax.set_xlabel("Fraction Downsampled", fontsize=8)
    ax.set_title("Fraction of newly recovered\npeaks with GCUAG motif", fontsize=8)
    ax.set_ylim(0,.50)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]

    sns.despine(ax=ax)
    ax.legend(fontsize=8)

In [ ]:
num_cols = 1
num_rows = 2

x = np.concatenate([motif_recovery_rep1.index.values, motif_recovery_rep2.index.values])
y = np.concatenate([motif_recovery_rep1.fraction_total_overlap_motif, motif_recovery_rep2.fraction_total_overlap_motif])

print scipy.stats.linregress(x,y)

with dataviz.Figure(os.path.join(img_dir, "fraction_of_ugcaug_recovered_cumsum.svg"), figsize=(2.5* num_rows, 2.5*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    ax.plot(motif_recovery_rep1.index, motif_recovery_rep1.fraction_total_overlap_motif, linewidth=2, alpha=.7, label="Rep 1")
    ax.plot(motif_recovery_rep2.index, motif_recovery_rep2.fraction_total_overlap_motif, linewidth=2, alpha=.7, label="Rep 2")

    ax.set_ylabel("Fraction containing motif", fontsize=8)
    ax.set_xlabel("Fraction Downsampled", fontsize=8)
    ax.set_title("Fraction RBFOX2 peaks with GCUAG motif", fontsize=8)
    ax.set_ylim(0,.50)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]

    sns.despine(ax=ax)
    ax.legend(loc=0, fontsize=8)
    


In [ ]:
motif_recovery_rep1_kmer = calculate_motif_recovery_kmer(rep1, "TGCATG")
motif_recovery_rep2_kmer = calculate_motif_recovery_kmer(rep2, "TGCATG")

In [ ]:
num_cols = 1
num_rows = 2
with dataviz.Figure(os.path.join(img_dir, "fraction_of_ugcaug_recovered.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(num_cols,num_rows,1)
    ax.plot(motif_recovery_rep1_kmer.index, motif_recovery_rep1_kmer.fraction_overlap_kmer, linewidth=3, alpha=.7, label="Rep 1")
    ax.plot(motif_recovery_rep2_kmer.index, motif_recovery_rep2_kmer.fraction_overlap_kmer, linewidth=3, alpha=.7, label="Rep 2")

    ax.set_ylabel("Fraction of new recovered Peaks with GCUAG Motif")
    ax.set_xlabel("Fraction Downsampled")
    ax.set_title("Quality of Recovered RBFOX2 Peaks")
    ax.set_ylim(0,.50)
    sns.despine(ax=ax)
    ax.legend()
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]


# Look at the splicing factors

In [ ]:
five_prime_ends = pybedtools.BedTool("/home/gpratt/clipper/clipper/data/regions/hg19_five_prime_ends.bed")
three_prime_ends = pybedtools.BedTool("/home/gpratt/clipper/clipper/data/regions/hg19_three_prime_ends.bed")

In [ ]:
colors = sns.color_palette("Set2", 2)
cell_types = ["HepG2", "K562"]
cell_type_colors = dict(zip(cell_types, colors))

In [ ]:
rep1 = merged_data.ix[('204', "RBFOX2", 'HepG2', 'rep1')]
rep2 = merged_data.ix[('204', "RBFOX2", 'HepG2', 'rep2')]

In [ ]:
def plot_both_cell_types(rbp, bedtool, site_name, total=False):
    num_cols = 1
    num_rows = 1
    
    if total:
        plot_y = "fraction_total_overlap_motif"
    else:
        plot_y = "fraction_overlap_motif"
    out_type = os.path.splitext(os.path.basename(bedtool.fn))[0]
    cell_types = merged_data.xs(rbp, level="RBP")
    
    total_flag = "total" if total else "fractional"
    x = np.array([])
    y = np.array([])
    with dataviz.Figure(os.path.join(img_dir, "fraction_of_{}_{}_{}_recovered.svg".format(rbp, out_type, total_flag)), figsize=(2.5* num_rows, 2.5*num_cols)) as fig:
        ax = fig.add_subplot(num_cols,num_rows,1)
        for name, group in cell_types.groupby(level=["uID", "Cell line", 'rep']):
            cell_type = name[1]
            motif_recovery_rep1 = calculate_motif_recovery(group, bedtool)
            ax.plot(motif_recovery_rep1.index.get_level_values(level="fraction"), 
                    motif_recovery_rep1[plot_y], 
                    linewidth=3, 
                    alpha=.7, 
                    label=" ".join(name),
                    color=cell_type_colors[cell_type]
                   )
            
            x = np.concatenate([x, motif_recovery_rep1.index.get_level_values(level="fraction").values])
            y = np.concatenate([y, motif_recovery_rep1[plot_y]])
        ax.set_ylabel("Fraction of recovered\npeaks overlapping with {}".format(site_name), fontsize=8)
        ax.set_xlabel("Peaks recovered at fraction", fontsize=8)
        ax.set_title("Fractions of peaks overlapping with {}".format(site_name), fontsize=8)
        ax.set_ylim(0,1)
        sns.despine(ax=ax)
        ax.legend(fontsize=8)
        [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(8) for tick in ax.get_yticklabels()]

    return scipy.stats.linregress(x, y)


In [ ]:
u2af1_3_ss_regressor = plot_both_cell_types("U2AF1", five_prime_ends, "3' splice site")

In [ ]:
u2af1_3_ss_regressor_total = plot_both_cell_types("U2AF1", five_prime_ends, "3' splice site", total=True)

In [ ]:
u2af1_3_ss_regressor = plot_both_cell_types("U2AF1", five_prime_ends, "3' splice site")

In [ ]:
u2af1_3_ss_regressor_total = plot_both_cell_types("U2AF1", five_prime_ends, "3' splice site", total=True)

In [ ]:
u2af1_5_ss_regressor = plot_both_cell_types("U2AF1", three_prime_ends, "5' splice site")

In [ ]:
u2af1_5_ss_regressor_total = plot_both_cell_types("U2AF1", three_prime_ends, "5' splice site", total=True)

In [ ]:
u2af2_3_ss_regressor = plot_both_cell_types("U2AF2", five_prime_ends, "3' splice site")

In [ ]:
u2af2_3_ss_regressor_total = plot_both_cell_types("U2AF2", five_prime_ends, "3' splice site", total=True)

In [ ]:
u2af2_5_ss_regressor = plot_both_cell_types("U2AF2", three_prime_ends, "5' splice site")

In [ ]:
u2af2_5_ss_regressor_total = plot_both_cell_types("U2AF2", three_prime_ends, "5' splice site", total=True)

In [ ]:
prpf8_5_ss_regressor = plot_both_cell_types("PRPF8", three_prime_ends, "5' splice site")

In [ ]:
prpf8_5_ss_regressor_total = plot_both_cell_types("PRPF8", three_prime_ends, "5' splice site", total=True)

In [ ]:
rmbm22_5_ss_regressor = plot_both_cell_types("RBM22", three_prime_ends, "5' splice site")

In [ ]:
rmbm22_5_ss_regressor_total = plot_both_cell_types("RBM22", three_prime_ends, "5' splice site", total=True)

In [ ]:
rmbm22_3_ss_regressor = plot_both_cell_types("RBM22", five_prime_ends, "3' splice site")

In [ ]:
rmbm22_3_ss_regressor_total = plot_both_cell_types("RBM22", five_prime_ends, "3' splice site", total=True)

In [ ]:
sf3b4_5_ss_regressor = plot_both_cell_types("SF3B4", three_prime_ends, "5' splice site")

In [ ]:
sf3b4_5_ss_regressor_total = plot_both_cell_types("SF3B4", three_prime_ends, "5' splice site", total=True)

In [ ]:
sf3b4_3_ss_regressor = plot_both_cell_types("SF3B4", five_prime_ends, "3' splice site")

In [ ]:
sf3b4_3_ss_regressor_total = plot_both_cell_types("SF3B4", five_prime_ends, "3' splice site", total=True)

In [ ]:
sf3b1_5_ss_regressor = plot_both_cell_types("SF3B1", three_prime_ends, "5' splice site")

In [ ]:
sf3b1_5_ss_regressor_total = plot_both_cell_types("SF3B1", three_prime_ends, "5' splice site", total=True)

In [ ]:
sf3b1_3_ss_regressor = plot_both_cell_types("SF3B1", five_prime_ends, "3' splice site")

In [ ]:
sf3b1_3_ss_regressor_total = plot_both_cell_types("SF3B1", five_prime_ends, "3' splice site", total=True)

In [ ]:
sf3b1_3_ss_regressor = plot_both_cell_types("SF3B1", five_prime_ends, "3' splice site")

In [ ]:
rbfox2_gcaug_regressor = plot_both_cell_types("RBFOX2", GCATG_hg19_motif, "UGCAUG Motifs")

In [ ]:
rbfox2_gcaug_regressor_total = plot_both_cell_types("RBFOX2", GCATG_hg19_motif, "UGCAUG Motifs", total=True)

In [ ]:
rbfox2_gcaug_regressor_total

To validate we looked at RBPs known to bind the splicesome, both 3' and 5' splice sites and looked at the fracton of peaks overlapping those regions

In [ ]:
def plot_both_cell_types_presentation(rbp, bedtool, site_name, total=False):
    num_cols = 1
    num_rows = 1
    
    if total:
        plot_y = "fraction_total_overlap_motif"
    else:
        plot_y = "fraction_overlap_motif"
    out_type = os.path.splitext(os.path.basename(bedtool.fn))[0]
    cell_types = merged_data.xs(rbp, level="RBP")
    
    total_flag = "total" if total else "fractional"
    x = np.array([])
    y = np.array([])
    with dataviz.Figure(os.path.join(img_dir, "fraction_of_{}_{}_{}_recovered_presentation.svg".format(rbp, out_type, total_flag)), figsize=(5* num_rows, 5*num_cols)) as fig:
        ax = fig.add_subplot(num_cols,num_rows,1)
        for name, group in cell_types.groupby(level=["uID", "Cell line", 'rep']):
            cell_type = name[1]
            motif_recovery_rep1 = calculate_motif_recovery(group, bedtool)
            ax.plot(motif_recovery_rep1.index.get_level_values(level="fraction"), 
                    motif_recovery_rep1[plot_y], 
                    linewidth=3, 
                    alpha=.7, 
                    label=" ".join(name),
                    color=cell_type_colors[cell_type]
                   )
            
            x = np.concatenate([x, motif_recovery_rep1.index.get_level_values(level="fraction").values])
            y = np.concatenate([y, motif_recovery_rep1[plot_y]])
        ax.set_ylabel("Fraction of peaks\nwith {}".format(site_name), fontsize=20)
        ax.set_xlabel("Fraction Downsampled", fontsize=20)
        ax.set_title("Fractions of peaks with {}".format(site_name), fontsize=20)
        ax.set_xticks(np.arange(0,1.1, .2))
        ax.set_ylim(0,1)
        sns.despine(ax=ax)
        ax.legend(fontsize=20)
        [tick.set_fontsize(20) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(20) for tick in ax.get_yticklabels()]

    return scipy.stats.linregress(x, y)


In [ ]:
rbfox2_gcaug_regressor_total = plot_both_cell_types_presentation("RBFOX2", GCATG_hg19_motif, "UGCAUG Motif", total=True)

In [ ]:
u2af2_3_ss_regressor_total = plot_both_cell_types_presentation("U2AF2", five_prime_ends, "3' splice site", total=True)

In [ ]:
def plot_both_cell_types_test(rbp, bedtool, site_name, total=False):
    num_cols = 1
    num_rows = 1
    
    if total:
        plot_y = "fraction_total_overlap_motif"
    else:
        plot_y = "fraction_overlap_motif"
    out_type = os.path.splitext(os.path.basename(bedtool.fn))[0]
    cell_types = merged_data.xs(rbp, level="RBP")
    
    total_flag = "total" if total else "fractional"
    x = np.array([])
    y = np.array([])
    with dataviz.Figure(os.path.join(img_dir, "fraction_of_{}_{}_{}_recovered.svg".format(rbp, out_type, total_flag)), figsize=(2.5* num_rows, 2.5*num_cols)) as fig:
        ax = fig.add_subplot(num_cols,num_rows,1)
        for name, group in cell_types.groupby(level=["uID", "Cell line", 'rep']):
            cell_type = name[1]
            motif_recovery_rep1 = calculate_motif_recovery(group, bedtool)
            motif_recovery_rep1['percent_decrease'] = (motif_recovery_rep1[plot_y] - motif_recovery_rep1[plot_y][-1]) / motif_recovery_rep1[plot_y][-1]
            #return motif_recovery_rep1
            ax.plot(motif_recovery_rep1.index.get_level_values(level="fraction"), 
                    motif_recovery_rep1['percent_decrease'], 
                    linewidth=3, 
                    alpha=.7, 
                    label=" ".join(name),
                    color=cell_type_colors[cell_type]
                   )
            
            x = np.concatenate([x, motif_recovery_rep1.index.get_level_values(level="fraction").values])
            y = np.concatenate([y, motif_recovery_rep1[plot_y]])
        ax.set_ylabel("Fraction of recovered\npeaks overlapping with {}".format(site_name), fontsize=8)
        ax.set_xlabel("Peaks recovered at fraction", fontsize=8)
        ax.set_title("Fractions of peaks overlapping with {}".format(site_name), fontsize=8)
        #ax.set_ylim(0,1)
        sns.despine(ax=ax)
        ax.legend(fontsize=8)
        [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(8) for tick in ax.get_yticklabels()]

    return scipy.stats.linregress(x, y)


In [ ]:
rbfox2_gcaug_regressor_total = plot_both_cell_types_test("RBFOX2", GCATG_hg19_motif, "UGCAUG Motifs", total=True)

In [ ]:
rbfox2_gcaug_regressor_total

# Look at conservation

loading old pickle files won't work because I collapse them too much

In [ ]:
conservation_out = "/projects/ps-yeolab3/encode/analysis/conservation_peaks"

In [ ]:
conservation = {}
grp = merged_data.groupby(level=["uID", "RBP", "Cell line", 'rep'])
for name, row in tqdm(list(total_bedtools.iterrows())):
    bedtool = pybedtools.BedTool(row.filtered_moderate)
    
    out_file = os.path.splitext(bedtool.fn)[0] + ".tab"
    if os.path.exists(out_file):
        result = pd.read_table(out_file, 
                               index_col=0,
                               header=None, 
                               names=['name', 'size', 'covered', 'sum', 'mean0', 'mean'])
    else:
        result = CLIP_analysis.bigWigAverageOverBed("/projects/ps-yeolab/genomes/hg19/hg19_phastcons.bw", bedtool)
    peaks_and_read_counts = get_recovered_peaks(grp.get_group(name))
    
    chrom_lst = []
    start_lst = []
    stop_lst = []

    for index in result.index:
        index = index.split("_")[-1]
        chrom, loc = index.split(":")
        start, stop = loc.split("-")


        chrom_lst.append(chrom)
        start_lst.append(int(start))
        stop_lst.append(int(stop))

    result['chrom'] = chrom_lst
    result['start'] = start_lst
    result['stop'] = stop_lst
    
    peaks_and_conservation = pd.merge(peaks_and_read_counts, result, 
             left_on=["chrom", "start", "stop"],
             right_on=["chrom", "start", "stop"],
             how="inner")
    
    out_name, ext = os.path.splitext(row.filtered_moderate)
    out_name = os.path.basename(out_name)
    out_name = os.path.join(conservation_out, out_name + ".conservation" + ext)
    
    peaks_and_conservation.to_csv(out_name)
    if len(peaks_and_read_counts) != len(peaks_and_conservation):
        print out_name
    
    peak_conservation = peaks_and_conservation.groupby("fraction")['mean']
    conservation[name] = pd.concat({'fractional_mean': peak_conservation.mean(),
                                    "cumsum_mean": peak_conservation.sum().cumsum() / peak_conservation.count().cumsum()},
                                   axis=1)

In [ ]:
conservation = pd.concat(conservation)
merged_data['fractional_conserved_mean'] = conservation['fractional_mean']
merged_data['cumsum_conserved_mean'] = conservation['cumsum_mean']

In [ ]:
grp = merged_data.groupby(level=["uID", "RBP", "Cell line", "rep"])
df_rep1 = grp.get_group(('204', 'RBFOX2', 'HepG2', "rep1"))
df_rep2 = grp.get_group(('204', 'RBFOX2', 'HepG2', "rep2"))

In [ ]:
num_cols = 1
num_rows = 1
out_type = os.path.splitext(os.path.basename(bedtool.fn))[0]
cell_types = merged_data.xs(rbp, level="RBP")
with dataviz.Figure(os.path.join(img_dir, "rbfox2_conservation.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(1,1,1)
    ax.plot(df_rep1.index.get_level_values("fraction"), df_rep1['fractional_conserved_mean'])
    ax.plot(df_rep2.index.get_level_values("fraction"), df_rep2['fractional_conserved_mean'])
    ax.set_xlabel("Fraction Downsampled")
    ax.set_ylabel("Mean Conservation")
    sns.despine(ax=ax)
    
x = np.concatenate([df_rep1.index.values, df_rep2.index.values])
y = np.concatenate([df_rep1.fractional_conserved_mean, df_rep2.fractional_conserved_mean])


In [ ]:
num_cols = 1
num_rows = 1
out_type = os.path.splitext(os.path.basename(bedtool.fn))[0]
cell_types = merged_data.xs(rbp, level="RBP")
with dataviz.Figure(os.path.join(img_dir, "rbfox2_conservation_cumsum.svg"), figsize=(2.5* num_rows, 2.5*num_cols)) as fig:
    ax = fig.add_subplot(1,1,1)
    ax.plot(df_rep1.index.get_level_values("fraction"), df_rep1['cumsum_conserved_mean'], label="Rep 1")
    ax.plot(df_rep2.index.get_level_values("fraction"), df_rep2['cumsum_conserved_mean'], label="Rep 2")
    ax.set_xlabel("Fraction Downsampled", fontsize=8)
    ax.set_ylabel("Mean Conservation", fontsize=8)
    ax.set_title("Conservation of RBFOX2 Peaks", fontsize=8)
    sns.despine(ax=ax)
    ax.legend(fontsize=8)
    [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
    [tick.set_fontsize(8) for tick in ax.get_yticklabels()]

x = np.concatenate([df_rep1.index.get_level_values(level="fraction"), df_rep2.index.get_level_values(level="fraction")])
y = np.concatenate([df_rep1.cumsum_conserved_mean, df_rep2.cumsum_conserved_mean])
    

In [ ]:
linregress = scipy.stats.linregress(x,y)

# I don't really know what I was doing here

Looks like I'm trying to get all the datasets with a significant relationship between conservation and sequencing depth

In [ ]:
grp = merged_data.groupby(level=["uID", "RBP", "Cell line"])
result = {}
for name, df in grp:
    x = np.array([])
    y = np.array([])
    for rep_name, rep_df in df.groupby(level="rep"):
        x = np.concatenate([x, rep_df.index.get_level_values(level="fraction").values])
        y = np.concatenate([y, rep_df.fractional_conserved_mean])   
    
    linregress = scipy.stats.linregress(x,y)
    result[name] = {"slope": linregress.slope,
                   "intercept": linregress.intercept,
                   "rvalue": linregress.rvalue,
                   'pvalue': linregress.pvalue,
                   'stderr': linregress.stderr,}
result = pd.DataFrame(result).T

In [ ]:
result_sig = result[result.pvalue < (.05 / 181)]

In [ ]:
result_sig

In [ ]:
num_cols = 1
num_rows = 1
out_type = os.path.splitext(os.path.basename(bedtool.fn))[0]
cell_types = merged_data.xs(rbp, level="RBP")
with dataviz.Figure(os.path.join(img_dir, "rbfox2_conservation.svg"), figsize=(4* num_rows, 4*num_cols)) as fig:
    ax = fig.add_subplot(1,1,1)
    ax.plot(df_rep1.index.get_level_values("fraction"), df_rep1['fractional_conserved_mean'])
    ax.plot(df_rep2.index.get_level_values("fraction"), df_rep2['fractional_conserved_mean'])
    ax.set_xlabel("Fraction Downsampled")
    ax.set_ylabel("Mean Conservation")
    sns.despine(ax=ax)

In [ ]:
def plot_both_cell_types_conservation(rbp):
    num_cols = 1
    num_rows = 1
    out_type = os.path.splitext(os.path.basename(bedtool.fn))[0]
    cell_types = merged_data.xs(rbp, level="RBP")
    
    x = np.array([])
    y = np.array([])
    with dataviz.Figure(os.path.join(img_dir, "{}_conservation_presentation.svg".format(rbp)), figsize=(4* num_rows, 4*num_cols)) as fig:
        ax = fig.add_subplot(num_cols,num_rows,1)
        for name, group in cell_types.groupby(level=["uID", "Cell line", 'rep']):
            cell_type = name[1]
            ax.plot(group.index.get_level_values(level="fraction"), 
                    group['cumsum_conserved_mean'], 
                    linewidth=3, 
                    alpha=.7, 
                    label=" ".join(name),
                    color=cell_type_colors[cell_type]
                   )
            
            x = np.concatenate([x, group.index.get_level_values(level="fraction").values])
            y = np.concatenate([y, group.cumsum_conserved_mean])
        ax.set_ylabel("Mean Conservation".format(), fontsize=18)
        ax.set_xlabel("Fraction Downsampled", fontsize=18)
        ax.set_title("Conservation of Peaks", fontsize=18)
        ax.set_ylim(0,1)
        sns.despine(ax=ax)
        ax.legend(fontsize=18)
        ax.set_xticks(np.arange(0,1.1, .2))
        [tick.set_fontsize(18) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(18) for tick in ax.get_yticklabels()]

    return scipy.stats.linregress(x, y)


In [ ]:
plot_both_cell_types_conservation("RBFOX2")

In [ ]:
plot_both_cell_types_conservation("U2AF2")

I don't really know what I'm doing here


# Plot without any filtering

In [ ]:
def fraction_bound_full(row):
    total_bedtool = total_bedtools.ix[row.name[:-1]].input_norm
    fractional_bedtool = row.input_norm
    
    total_bedtool = pybedtools.BedTool(total_bedtool)
    fractional_bedtool = pybedtools.BedTool(fractional_bedtool)
    return len(total_bedtool.intersect(fractional_bedtool, u=True, s=True, sorted=True)) / float(len(total_bedtool))
 
def fraction_of_fraction_in_total_full(row):
    total_bedtool = total_bedtools.ix[row.name[:-1]].input_norm
    fractional_bedtool = row.input_norm

    total_bedtool = pybedtools.BedTool(total_bedtool)
    fractional_bedtool = pybedtools.BedTool(fractional_bedtool)
    return len(fractional_bedtool.intersect(total_bedtool, u=True, s=True, sorted=True)) / float(len(fractional_bedtool))

def fraction_not_in_total_full(row):
    total_bedtool = total_bedtools.ix[row.name[:-1]].input_norm
    fractional_bedtool = row.input_norm

    total_bedtool = pybedtools.BedTool(total_bedtool)
    fractional_bedtool = pybedtools.BedTool(fractional_bedtool)
    return len(fractional_bedtool.intersect(total_bedtool, v=True, s=True, sorted=True)) / float(len(fractional_bedtool))

In [ ]:
merged_data['fraction_overlap_full'] = merged_data.apply(fraction_bound_full, axis=1)
merged_data['fraction_not_in_total_full'] = merged_data.apply(fraction_not_in_total_full, axis=1)

In [ ]:
total_datasets = len(result_df.groupby(level=0))
num_cols = 4
num_rows = (total_datasets / 4) + 1
count = 0

with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "downsampling_peaks.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    for uID, df in result_df.groupby(level=0):
        count += 1
        ax = fig.add_subplot(num_rows, num_cols, count)

        for rep, dataset_df in df.groupby(level=1):
            name = "{} {}".format(uID, fully_downsampled_data[fully_downsampled_data.uID == uID].RBP.values[0])
            ax.plot(dataset_df[reads], dataset_df.fraction_of_peaks, 
                    label=name, 
                    alpha=.7, 
                    linewidth=3)
            ax.xaxis.set_major_formatter(xfmt)
            ax.set_xlabel("Reads (M)")
            ax.set_ylabel("Fraction Overlapped")
            ax.set_ylim(0,1)
            sns.despine(ax=ax)
        ax.legend(fontsize=18)

In [ ]:
total_datasets = len(fraction_not_in_total_unfiltered_df.groupby(level=0))
num_cols = 4
num_rows = (total_datasets / 4) + 1
count = 0

with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "downsampling_peaks.svg"), figsize=(4* num_cols, 4*num_rows)) as fig:
    for uID, df in fraction_not_in_total_unfiltered_df.groupby(level=0):
        count += 1
        ax = fig.add_subplot(num_rows, num_cols, count)

        for rep, dataset_df in df.groupby(level=1):
            name = "{} {}".format(uID, fully_downsampled_data[fully_downsampled_data.uID == uID].RBP.values[0])
            ax.plot(dataset_df[reads], dataset_df.fraction_of_peaks, 
                    label=name, 
                    alpha=.7, 
                    linewidth=3)
            ax.xaxis.set_major_formatter(xfmt)
            ax.set_xlabel("Reads (M)")
            ax.set_ylabel("Fraction Overlapped")
            ax.set_ylim(0,1)
            sns.despine(ax=ax)
        ax.legend(fontsize=18)

# So the unfiltered peaks appear to overlap nicely.  Why do they go away? I'll look at RBFOX2 as a case study.


I want peaks that are siginifant in 09, but not significant in full, but exist in full

In [ ]:
print unfiltered_full_fn
print unfiltered_downsampled_fn
print get_full_from_annotated(unfiltered_full_fn)
print get_dowmsample_from_annotated(unfiltered_downsampled_fn)

In [ ]:
unfiltered_full_fn = fully_downsampled_data.ix[1]['uID_01.basedon_uID_01.peaks.l2inputnormnew.bed.compressed.bed']
unfiltered_downsampled_fn = fully_downsampled_data.ix[1]['CLIP_rep1_09']

unfiltered_full_peaks = pybedtools.BedTool(unfiltered_full_fn)

filtered_full_peaks = pybedtools.BedTool(make_and_filter_clipper_moderate(unfiltered_full_fn))
filtered_downsampled_peaks = pybedtools.BedTool(make_and_filter_clipper_moderate(unfiltered_downsampled_fn))

not_in_full_dataset = filtered_downsampled_peaks.intersect(filtered_full_peaks, v=True, s=True)
peaks_filtered_from_full = not_in_full_dataset.intersect(unfiltered_full_peaks, s=True, wo=True)

In [ ]:
peaks_filtered_from_full_df = peaks_filtered_from_full.to_dataframe(names=["chrom_subset", "start_subset", 
                                      "stop_subset", "name_subset", 
                                      "score_subset", "strand_subset",
                                      "chrom_full", "start_full", 
                                      "stop_full", "name_full", 
                                      "score_full", "strand_full", "annotation_full",
                                      "gene_full", "foo_full"])

In [ ]:
def get_dowmsample_from_annotated(fn):
    stripped_fn = ".".join(fn.split(".")[:-2])
    return stripped_fn + ".bed.full"

In [ ]:
print get_dowmsample_from_annotated(unfiltered_downsampled_fn)
print os.path.exists(get_dowmsample_from_annotated(unfiltered_downsampled_fn))

In [ ]:
full_counts = pd.read_table(get_full_from_annotated(unfiltered_full_fn), 
                           names=["chrom", "start", "stop", "full_name", "ip_reads", "input_reads", "p_val", "fold_change", "something", "enrichment", "log10_p_val", "log2_fold_change"])

downsampled_counts = pd.read_table(get_dowmsample_from_annotated(unfiltered_downsampled_fn), 
                           names=["chrom", "start", "stop", "full_name", "ip_reads", "input_reads", "p_val", "fold_change", "something", "enrichment", "log10_p_val", "log2_fold_change"])

In [ ]:
merged_with_full = pd.merge(peaks_filtered_from_full_df, full_counts, 
                            left_on=["chrom_full", "start_full", "stop_full"], 
                            right_on=["chrom", "start", "stop"], 
                            how="left",
                            suffixes=["bedtool", "full"])

merged_full_downsampled = pd.merge(merged_with_full, downsampled_counts,
                                   left_on=["chrom_subset", "start_subset", "stop_subset"], 
                                   right_on=["chrom", "start", "stop"], 
                                   how="left",
                                   suffixes=['_full_count', '_downsampled_count'])

In [ ]:
subset_length = merged_full_downsampled.stop_subset - merged_full_downsampled.start_subset
full_length = merged_full_downsampled.stop_full - merged_full_downsampled.start_full

num_rows = 2
num_cols = 3
with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "peak_downsampling_figure.svg"), figsize=(5* num_cols, 5*num_rows)) as fig:
        ax = fig.add_subplot(num_rows, num_cols, 1)
        ax.scatter(subset_length, full_length)
        ax.set_xlabel("Subset Peak Length (bp)")
        ax.set_ylabel("Full Peak Length (bp)")
        ax.set_title("Overlapping Peak Lengths")
        sns.despine(ax=ax)

        ax = fig.add_subplot(num_rows, num_cols, 2)
        ax.scatter(merged_full_downsampled.log10_p_val_downsampled_count, merged_full_downsampled.log10_p_val_full_count)
        ax.set_xlabel("Downsample p-value")
        ax.set_ylabel("Full p-value")
        ax.set_title("Log 10 P-value")
        ax.axvline(3)
        ax.axhline(3)

        sns.despine(ax=ax)
        
        ax = fig.add_subplot(num_rows, num_cols, 3)
        ax.scatter(merged_full_downsampled.log2_fold_change_downsampled_count, merged_full_downsampled.log2_fold_change_full_count)
        ax.set_xlabel("Downsample Fold Change")
        ax.set_ylabel("Full Fold Change")
        ax.set_title("Fold Changes")
        ax.axvline(3)
        ax.axhline(3)
        sns.despine(ax=ax)
        
        ax = fig.add_subplot(num_rows, num_cols, 4)
        ax.scatter(np.log10(merged_full_downsampled.ip_reads_downsampled_count), 
                   np.log10(merged_full_downsampled.ip_reads_full_count))
        ax.set_xlabel("Downsample  IP Read Count")
        ax.set_ylabel("Full IP Read Count")
        ax.set_title("IP Read Count")
        ax.axvline(3)
        ax.axhline(3)
        sns.despine(ax=ax)
        
        ax = fig.add_subplot(num_rows, num_cols, 5)
        ax.scatter(np.log10(merged_full_downsampled.input_reads_downsampled_count), 
                   np.log10(merged_full_downsampled.input_reads_full_count))
        ax.set_xlabel("Downsample Input Read Count")
        ax.set_ylabel("Full Input Read Count")
        ax.set_title("Input Read Count")
        ax.axvline(3)
        ax.axhline(3)
        sns.despine(ax=ax)

In [ ]:
len(merged_full_downsampled)

In [ ]:
len(merged_full_downsampled[merged_full_downsampled.ip_reads_downsampled_count > merged_full_downsampled.ip_reads_full_count])

In [ ]:
scipy.stats.linregress(subset_length, full_length)

Conclusion from here is that the peak lengths don't really change too much.  R2 isn't great, but its alright?

In [ ]:
num_rows = 1
num_cols = 1
with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "peak_downsampling_figure.svg"), figsize=(5* num_cols, 5*num_rows)) as fig:
        ax = fig.add_subplot(num_rows, num_cols, 1)


In [ ]:
total_datasets = len(result_df.groupby(level=0))
num_rows = 1
num_cols = 1
count = 0

with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "peak_downsampling_figure.svg"), figsize=(5* num_cols, 5*num_rows)) as fig:
        count += 1
        ax = fig.add_subplot(num_rows, num_cols, count)

#         for rep, dataset_df in foo.get_group("244").groupby(level=1):
#             ax.plot(dataset_df[reads], dataset_df.fraction_of_peaks, label=fully_downsampled_data[fully_downsampled_data.uID == "244"].RBP.values[0], color='b')
        uid = "228"
        for rep, dataset_df in foo.get_group(uid).groupby(level=1):
            ax.plot(dataset_df[reads], dataset_df.fraction_of_peaks, label=fully_downsampled_data[fully_downsampled_data.uID == uid].RBP.values[0],
                    alpha=.7, linewidth=3, color='r')
            break

        uid = "204"
        for rep, dataset_df in foo.get_group(uid).groupby(level=1):
            ax.plot(dataset_df.reads, dataset_df.fraction_of_peaks, label=fully_downsampled_data[fully_downsampled_data.uID == uid].RBP.values[0], 
                    alpha=.7, linewidth=3, color='b')
            break
            
#         for rep, dataset_df in foo.get_group("262").groupby(level=1):
#             ax.plot(dataset_df[reads], dataset_df.fraction_of_peaks, label=fully_downsampled_data[fully_downsampled_data.uID == "262"].RBP.values[0], 
#                     alpha=.7, linewidth=3, color='y')
#             break

        ax.set_xlabel("Number of reads", fontsize=18)
        ax.set_ylabel("Fraction of peaks overlapping all peaks", fontsize=18)
        [tick.set_fontsize(14) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(14) for tick in ax.get_yticklabels()]
        sns.despine(ax=ax)
        ax.legend()
        ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))


In [ ]:
total_datasets = len(result_df.groupby(level=0))
num_rows = 1
num_cols = 1
count = 0

with dataviz.Figure(os.path.join("/home/gpratt/Dropbox/encode_integration/qc_work/", "peak_downsampling_figure_slbp.svg"), figsize=(5* num_cols, 5*num_rows)) as fig:
        count += 1
        ax = fig.add_subplot(num_rows, num_cols, count)
       
        for rep, dataset_df in foo.get_group("262").groupby(level=1):
            ax.plot(dataset_df[reads], dataset_df.fraction_of_peaks, label=fully_downsampled_data[fully_downsampled_data.uID == "262"].RBP.values[0], 
                    alpha=.7, linewidth=3, color='y')
            break

#         for rep, dataset_df in foo.get_group("243").groupby(level=1):
#             ax.plot(dataset_df[reads], dataset_df.fraction_of_peaks, label=fully_downsampled_data[fully_downsampled_data.uID == "243"].RBP.values[0], 
#                     alpha=.7, linewidth=3, color='g')
#             break

        ax.set_xlabel("Number of reads", fontsize=18)
        ax.set_ylabel("Fraction of peaks overlapping all peaks", fontsize=18)
        ax.set_ylim(0,1)
        [tick.set_fontsize(8) for tick in ax.get_xticklabels()]
        [tick.set_fontsize(8) for tick in ax.get_yticklabels()]
        sns.despine(ax=ax)
        ax.legend()
        ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))


# Try more accurate model of data